In [3]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import json

import warnings
warnings.filterwarnings('ignore')

In [5]:
#lesum inn gögn af epl offical og understat
import pickle

pick_read = open('../data/understat_epl_ready.pickle','rb')
understat_epl_ready = pickle.load(pick_read)
pick_read.close()

pick_read = open('../data/Epl_official_df_updated.pickle','rb')
Epl_official_ready = pickle.load(pick_read)
pick_read.close()


In [6]:
#lesum inn odds af football-data.co.uk
csvs = ['https://www.football-data.co.uk/mmz4281/1718/E0.csv',
        'https://www.football-data.co.uk/mmz4281/1819/E0.csv',
        'https://www.football-data.co.uk/mmz4281/1920/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2021/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2122/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2223/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2324/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2425/E0.csv']
odds = pd.DataFrame()
for c in csvs:
    odds = pd.concat([odds, pd.read_csv(c)])
odds.to_csv("football_odds.csv.gzip", index=False, compression='gzip') #vista saman
odds.shape

(3040, 153)

In [7]:
odds_df = odds[['Date','HomeTeam','AwayTeam', 'HTR','HS', 'AS', 'HST', 'AST', 'HC', 'AC',
                'HF','AF', 'B365H','B365D','B365A' ]]
odds_df.columns = ['Date','Home_Team','Away_Team', 'Half Time Result','HomeShots','AwayShots','HomeShotOT','AwayShotsOT',
                   'HomeFouls','AwayFouls','HomeCorners', 'AwayCorners','OddsH','OddsD','OddsA']
odds_df.head()

,Date,Home_Team,Away_Team,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA
0,11/08/2017,Arsenal,Leicester,D,27,6,10,3,9,4,9,12,1.53,4.5,6.50
1,12/08/2017,Brighton,Man City,D,6,14,2,4,3,10,6,9,11.00,5.5,1.33
2,12/08/2017,Chelsea,Burnley,A,19,10,6,5,8,5,16,11,1.25,6.5,15.00
3,12/08/2017,Crystal Palace,Huddersfield,A,14,8,4,6,12,9,7,19,1.83,3.6,5.00
4,12/08/2017,Everton,Stoke,H,9,9,4,1,6,7,13,10,1.70,3.8,5.75


In [8]:
Epl_official_ready.rename(columns={'Match_Date': 'Date'}, inplace=True)

In [9]:
#þurfum bara odds úr odds_df, lögum einnig date
odds_df.columns

Index(['Date', 'Home_Team', 'Away_Team', 'Half Time Result', 'HomeShots',
       'AwayShots', 'HomeShotOT', 'AwayShotsOT', 'HomeFouls', 'AwayFouls',
       'HomeCorners', 'AwayCorners', 'OddsH', 'OddsD', 'OddsA'],
      dtype='object')

In [10]:
odds_df = odds_df.drop(['Half Time Result', 'HomeShots',
       'AwayShots', 'HomeShotOT', 'AwayShotsOT', 'HomeFouls', 'AwayFouls',
       'HomeCorners', 'AwayCorners'], axis=1)
odds_df.head()

,Date,Home_Team,Away_Team,OddsH,OddsD,OddsA
0,11/08/2017,Arsenal,Leicester,1.53,4.5,6.50
1,12/08/2017,Brighton,Man City,11.00,5.5,1.33
2,12/08/2017,Chelsea,Burnley,1.25,6.5,15.00
3,12/08/2017,Crystal Palace,Huddersfield,1.83,3.6,5.00
4,12/08/2017,Everton,Stoke,1.70,3.8,5.75


In [11]:
odds_df['Date'] = pd.to_datetime(odds_df['Date'], format='%d/%m/%Y', errors='coerce')
odds_df['Date'] = odds_df['Date'].dt.strftime('%Y/%m/%d')
odds_df.head()

,Date,Home_Team,Away_Team,OddsH,OddsD,OddsA
0,2017/08/11,Arsenal,Leicester,1.53,4.5,6.50
1,2017/08/12,Brighton,Man City,11.00,5.5,1.33
2,2017/08/12,Chelsea,Burnley,1.25,6.5,15.00
3,2017/08/12,Crystal Palace,Huddersfield,1.83,3.6,5.00
4,2017/08/12,Everton,Stoke,1.70,3.8,5.75


In [13]:
#skoða hvort lið séu með mismunandi nöfn milli gagnasafna
a = (odds_df.Home_Team.unique())
b = (understat_epl_ready.Home_Team.unique())
c = (Epl_official_ready.Home_Team.unique())
print(sorted(a))
print(sorted(b))
print(sorted(c))


['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield', 'Ipswich', 'Leeds', 'Leicester', 'Liverpool', 'Luton', 'Man City', 'Man United', 'Newcastle', 'Norwich', "Nott'm Forest", 'Sheffield United', 'Southampton', 'Stoke', 'Swansea', 'Tottenham', 'Watford', 'West Brom', 'West Ham', 'Wolves']
['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield', 'Ipswich', 'Leeds', 'Leicester', 'Liverpool', 'Luton', 'Manchester City', 'Manchester United', 'Newcastle United', 'Norwich', 'Nottingham Forest', 'Sheffield United', 'Southampton', 'Stoke', 'Swansea', 'Tottenham', 'Watford', 'West Bromwich Albion', 'West Ham', 'Wolverhampton Wanderers']
['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield', 'Ipswich',

In [14]:
replace_values = {'Man City' : 'Manchester City', 'Man United' : 'Manchester United', 'Man Utd' : 'Manchester United', 'Newcastle' : 'Newcastle United',
                  'Nott\'m Forest': 'Nottingham Forest',  'West Brom' : 'West Bromwich Albion', 'Sheffield Utd' : 'Sheffield United',
                  'Sheff Utd' : 'Sheffield United', 'Spurs' : 'Tottenham', 'Wolves' : 'Wolverhampton Wanderers'}

odds_df['Home_Team'] = odds_df['Home_Team'].map(replace_values).fillna(odds.HomeTeam)
odds_df['Away_Team'] = odds_df['Away_Team'].map(replace_values).fillna(odds.AwayTeam)
Epl_official_ready['Home_Team'] = Epl_official_ready['Home_Team'].map(replace_values).fillna(Epl_official_ready.Home_Team)
Epl_official_ready['Away_Team'] = Epl_official_ready['Away_Team'].map(replace_values).fillna(Epl_official_ready.Away_Team)

In [15]:
#breyta í prósentur
for x in ['OddsH','OddsD','OddsA']:
    odds_df[x]=1/odds_df[x]

odds_df.shape

(3040, 6)

In [16]:
odds_df.head()

,Date,Home_Team,Away_Team,OddsH,OddsD,OddsA
0,2017/08/11,Arsenal,Leicester,0.653595,0.222222,0.153846
1,2017/08/12,Brighton,Manchester City,0.090909,0.181818,0.751880
2,2017/08/12,Chelsea,Burnley,0.800000,0.153846,0.066667
3,2017/08/12,Crystal Palace,Huddersfield,0.546448,0.277778,0.200000
4,2017/08/12,Everton,Stoke,0.588235,0.263158,0.173913


In [17]:
#mergea saman

test = pd.merge(left=understat_epl_ready, right=odds_df,
                  left_on= ['Date','Home_Team', 'Away_Team'],
                   right_on=['Date','Home_Team', 'Away_Team'],
                  how='left')




In [18]:
test.head()

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,OddsH,OddsD,OddsA
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455,0.653595,0.222222,0.153846
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444,0.090909,0.181818,0.751880
2,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,14.7143,0.166667,0.238095,0.617284
3,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,12.0000,0.416667,0.303030,0.303030
4,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,12.8824,0.617284,0.250000,0.153846


In [19]:
Epl_data_ready = pd.merge(left=test, right=Epl_official_ready,
                  left_on= ['Date','Home_Team', 'Away_Team'],
                   right_on=['Date','Home_Team', 'Away_Team'],
                  how='left')

In [20]:
Epl_data_ready.head()

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,...,Away_Blocks,Home_Interceptions,Away_Interceptions,Home_Clearances,Away_Clearances,Home_Duels_Won,Away_Duels_Won,Home_Aerial_Duels_Won,Away_Aerial_Duels_Won,Season
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,...,8,13,11,30,24,69,48,18,18,17/18
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,...,7,12,9,39,8,38,35,13,13,17/18
2,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,...,2,11,11,28,22,60,64,22,25,17/18
3,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,...,6,14,5,26,17,34,52,12,18,17/18
4,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,...,13,12,8,11,38,47,42,17,18,17/18


In [ ]:
#nú má framkvæma feature extration og reikna út lýsandi gögn fyrir ml líkönin
#byrjum á að droppa dálkum sem lýsa því sama sem eru ekki nothæfir
#gaman væri að setja inn einhverskonar rating system svipað elo

In [21]:
pd.set_option('display.max_columns', None)

In [22]:
Epl_data_ready.head(2)

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,OddsH,OddsD,OddsA,Matchweek,Home_Possession,Away_Possession,Home_Total_Shots,Away_Total_Shots,Home_Shots_On_Target,Away_Shots_On_Target,Home_Big_Chances,Away_Big_Chances,Home_Passes,Away_Passes,Home_Corners,Away_Corners,Home_Saves,Away_Saves,Home_Shots_Off_Target,Away_Shots_Off_Target,Home_Shots_Inside_the_Box,Away_Shots_Inside_the_Box,Home_Shots_Outside_the_Box,Away_Shots_Outside_the_Box,Home_Big_Chances_Created,Away_Big_Chances_Created,Home_Total_Crosses_(%_Completed),Away_Total_Crosses_(%_Completed),Home_Total_Passes,Away_Total_Passes,Home_Long_Passes_(%_Completed),Away_Long_Passes_(%_Completed),Home_Through_Balls_(%_Completed),Away_Through_Balls_(%_Completed),Home_Touches,Away_Touches,Home_Touches_in_the_opposition_box,Away_Touches_in_the_opposition_box,Home_Tackles_Won,Away_Tackles_Won,Home_Blocks,Away_Blocks,Home_Interceptions,Away_Interceptions,Home_Clearances,Away_Clearances,Home_Duels_Won,Away_Duels_Won,Home_Aerial_Duels_Won,Away_Aerial_Duels_Won,Season
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455,0.653595,0.222222,0.153846,Matchweek 1,70%,30%,27,6,10,3,1,2,632 (85%),263 (63%),9,4,0,6,9,3,3,0,12,1,4,2,20 (30%),18 (22%),632 (85%),263 (63%),55 (67%),70 (46%),2 (50%),2 (50%),859,459,30,11,17 (74%),8 (47%),9,8,13,11,30,24,69,48,18,18,17/18
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444,0.090909,0.181818,0.751880,Matchweek 1,21.8%,78.2%,6,14,2,4,0,1,213 (61%),768 (90%),3,10,3,2,2,5,2,1,2,8,0,3,8 (25%),27 (15%),213 (61%),768 (90%),49 (37%),52 (65%),0 (0%),3 (67%),375,918,9,28,8 (80%),6 (60%),7,7,12,9,39,8,38,35,13,13,17/18


In [23]:
#brjótum upp dálka sem eru með %completed eða slíkt
def split_stat_column(df, column_name):
    base_name = column_name.replace('_(%_Completed)', '')

    #Skipta upp dálkagildum
    extracted = df[column_name].str.extract(r'(\d+)\s*\((\d+)%\)')

    #breytum % í float
    df[base_name] = pd.to_numeric(extracted[0], errors='coerce')
    df[f"{base_name}_Completed"] = pd.to_numeric(extracted[1], errors='coerce') / 100

    #henda gamla dálknum
    df.drop(columns=[column_name], inplace=True)


In [24]:
cols_to_split = [
    'Home_Passes',
    'Away_Passes',
    'Home_Total_Crosses_(%_Completed)',
    'Away_Total_Crosses_(%_Completed)',
    'Home_Total_Passes',
    'Away_Total_Passes',
    'Home_Long_Passes_(%_Completed)',
    'Away_Long_Passes_(%_Completed)',
    'Home_Through_Balls_(%_Completed)',
    'Away_Through_Balls_(%_Completed)',
    'Home_Tackles_Won',
    'Away_Tackles_Won',
]

for col in cols_to_split:
    split_stat_column(Epl_data_ready, col)

In [25]:
cols = ['Home_Possession', 'Away_Possession']
for col in cols:
    Epl_data_ready[col] = pd.to_numeric(Epl_data_ready[col].str.rstrip('%'), errors='coerce') / 100


In [ ]:
#núna skulum við losa okkur við NAN gildi og skipta þeim
#allar raðir með nan
nan_rows = Epl_data_ready[Epl_data_ready.isnull().T.any()]

In [27]:
nan_rows

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,OddsH,OddsD,OddsA,Matchweek,Home_Possession,Away_Possession,Home_Total_Shots,Away_Total_Shots,Home_Shots_On_Target,Away_Shots_On_Target,Home_Big_Chances,Away_Big_Chances,Home_Corners,Away_Corners,Home_Saves,Away_Saves,Home_Shots_Off_Target,Away_Shots_Off_Target,Home_Shots_Inside_the_Box,Away_Shots_Inside_the_Box,Home_Shots_Outside_the_Box,Away_Shots_Outside_the_Box,Home_Big_Chances_Created,Away_Big_Chances_Created,Home_Touches,Away_Touches,Home_Touches_in_the_opposition_box,Away_Touches_in_the_opposition_box,Home_Blocks,Away_Blocks,Home_Interceptions,Away_Interceptions,Home_Clearances,Away_Clearances,Home_Duels_Won,Away_Duels_Won,Home_Aerial_Duels_Won,Away_Aerial_Duels_Won,Season,Home_Passes_Completed,Away_Passes_Completed,Home_Total_Crosses,Home_Total_Crosses_Completed,Away_Total_Crosses,Away_Total_Crosses_Completed,Home_Total_Passes_Completed,Away_Total_Passes_Completed,Home_Long_Passes,Home_Long_Passes_Completed,Away_Long_Passes,Away_Long_Passes_Completed,Home_Through_Balls,Home_Through_Balls_Completed,Away_Through_Balls,Away_Through_Balls_Completed,Home_Tackles_Won_Completed,Away_Tackles_Won_Completed
4,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,12.8824,0.617284,0.250000,0.153846,Matchweek 1,0.596,0.404,29,4,2,0,NaN,NaN,13,0,0,2,16,2,NaN,NaN,13,1,2,0,707,532,38,7,15,13,12,8,11,38,47,42,17,18,17/18,0.83,0.78,35.0,0.34,8.0,0.25,0.83,0.78,46.0,0.39,71.0,0.49,1.0,0.0,0.0,0.00,0.83,0.73
11,2017/08/19,Burnley,West Bromwich Albion,0,1,1.37242,0.653058,4,0,5.5000,14.0000,0.380228,0.312500,0.333333,Matchweek 2,0.678,0.322,20,8,0,1,NaN,NaN,5,5,NaN,NaN,15,5,NaN,NaN,9,3,2,0,665,405,20,9,7,8,7,8,33,36,52,44,24,20,17/18,0.79,0.56,33.0,0.24,13.0,0.31,0.79,0.56,87.0,0.46,73.0,0.33,0.0,0.0,0.0,0.00,0.89,0.33
14,2017/08/19,Stoke,Arsenal,1,0,0.692644,1.40658,4,15,24.0667,5.1053,0.222222,0.263158,0.540541,Matchweek 2,0.229,0.771,11,18,4,6,NaN,NaN,2,9,6,3,5,6,3,5,2,9,0,2,395,908,11,37,8,8,8,7,35,9,54,45,18,9,17/18,0.64,0.87,10.0,0.50,24.0,0.38,0.64,0.87,62.0,0.37,41.0,0.54,2.0,0.5,7.0,0.29,0.58,0.73
15,2017/08/19,Southampton,West Ham,3,2,2.1424,1.53269,14,1,4.8519,23.4615,0.571429,0.263158,0.190476,Matchweek 2,0.661,0.339,14,16,5,8,2,2,7,2,6,2,6,6,1,5,7,3,NaN,NaN,725,455,26,14,5,5,8,12,19,38,39,61,16,25,17/18,0.86,0.67,26.0,0.15,18.0,0.56,0.86,0.67,43.0,0.37,79.0,0.43,0.0,0.0,1.0,1.00,0.70,0.75
17,2017/08/20,Huddersfield,Newcastle United,1,0,0.267008,0.973322,2,5,7.7857,13.8421,0.400000,0.307692,0.312500,Matchweek 2,0.527,0.473,7,13,3,5,NaN,NaN,7,3,5,2,3,3,0,3,4,4,0,1,674,611,11,19,6,7,18,7,29,27,62,50,24,12,17/18,0.77,0.71,20.0,0.05,14.0,0.29,0.77,0.71,69.0,0.45,81.0,0.43,1.0,0.0,0.0,0.00,0.54,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2992,2025/04/26,Chelsea,Everton,1,0,0.768313,0.916838,2,0,10.5769,14.6190,0.645161,0.250000,0.166667,Matchweek 34,0.547,0.453,10,6,7,3,NaN,NaN,10,2,3,6,2,1,4,1,5,2,0,2,703,600,19,11,3,3,7,7,20,21,62,41,13,8,24/25,0.86,0.84,20.0,0.20,18.0,0.33,0.86,0.84,56.0,0.43,51.0,0.47,1.0,0.0,0.0,0.00,0.61,0.76
3012,2025/05/10,Southampton,Manchester City,0,0,0.136995,1.58463,3,24,17.2500,7.3636,0.090909,0.133333,0.833333,Matchweek 36,0.282,0.718,2,26,0,5,NaN,NaN,1,15,4,0,0,8,0,3,0,12,NaN,NaN,444,818,7,76,16,17,11,3,52,8,40,51,17,17,24/25,0.76,0.89,5.0,0.20,44.0,0.23,0.76,0.89,57.0,0.37,26.0,0.58,0.0,0.0,1.0,1.00,0.62,0.50
3022,2025/05/18,Arsenal,Newcastle United,1,0,1.15958,2.24634,3,8,11.1111,9.3684,0.512821,0.270270,0.270270,Matchweek 37,0.514,0.486,12,14,4,5,NaN,NaN,10,7,5,3,3,3,2,4,5,4,1,2,612,570,32,32,10,14,6,6,30,26,49,51,15,19,24/25,0.83,0.83,15.0,0.33,27.0,0.15,0.83,0.83,47.0,0.38,47.0,0.40,1.0,0.0,1.0,1.00,0.53,0.60
3023,2025/05/18,Leicester,Ipswich,2,0,0.78

In [34]:
#hvaða dálkum má halda og hverjir eru ekki nothæfir
test = nan_rows[nan_rows.columns[nan_rows.isnull().any()]].isnull().sum()
test

Home_Possession                        12
Away_Possession                        12
Home_Total_Shots                       11
Away_Total_Shots                       11
Home_Shots_On_Target                   11
Away_Shots_On_Target                   11
Home_Big_Chances                      617
Away_Big_Chances                      617
Home_Corners                           11
Away_Corners                           11
Home_Saves                             23
Away_Saves                             23
Home_Shots_Off_Target                  11
Away_Shots_Off_Target                  11
Home_Shots_Inside_the_Box              81
Away_Shots_Inside_the_Box              81
Home_Shots_Outside_the_Box             11
Away_Shots_Outside_the_Box             11
Home_Big_Chances_Created              199
Away_Big_Chances_Created              199
Home_Touches                           11
Away_Touches                           11
Home_Touches_in_the_opposition_box     11
Away_Touches_in_the_opposition_box

In [35]:
#sjáum að þær eru töluvert margar, losum okkur við home og away saves og big chances sem virðist
#ekki hafa verið skráð alltaf í stats á premierleague.com
Epl_data_ready = Epl_data_ready.drop(['Home_Big_Chances', 'Away_Big_Chances',
                                      'Home_Big_Chances_Created', 'Away_Big_Chances_Created'], axis=1)
Epl_data_ready.head()

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,OddsH,OddsD,OddsA,Matchweek,Home_Possession,Away_Possession,Home_Total_Shots,Away_Total_Shots,Home_Shots_On_Target,Away_Shots_On_Target,Home_Corners,Away_Corners,Home_Saves,Away_Saves,Home_Shots_Off_Target,Away_Shots_Off_Target,Home_Shots_Inside_the_Box,Away_Shots_Inside_the_Box,Home_Shots_Outside_the_Box,Away_Shots_Outside_the_Box,Home_Touches,Away_Touches,Home_Touches_in_the_opposition_box,Away_Touches_in_the_opposition_box,Home_Blocks,Away_Blocks,Home_Interceptions,Away_Interceptions,Home_Clearances,Away_Clearances,Home_Duels_Won,Away_Duels_Won,Home_Aerial_Duels_Won,Away_Aerial_Duels_Won,Season,Home_Passes_Completed,Away_Passes_Completed,Home_Total_Crosses,Home_Total_Crosses_Completed,Away_Total_Crosses,Away_Total_Crosses_Completed,Home_Total_Passes_Completed,Away_Total_Passes_Completed,Home_Long_Passes,Home_Long_Passes_Completed,Away_Long_Passes,Away_Long_Passes_Completed,Home_Through_Balls,Home_Through_Balls_Completed,Away_Through_Balls,Away_Through_Balls_Completed,Home_Tackles_Won_Completed,Away_Tackles_Won_Completed
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455,0.653595,0.222222,0.153846,Matchweek 1,0.700,0.300,27,6,10,3,9,4,0,6,9,3,3,0,12,1,859,459,30,11,9,8,13,11,30,24,69,48,18,18,17/18,0.85,0.63,20.0,0.30,18.0,0.22,0.85,0.63,55.0,0.67,70.0,0.46,2.0,0.5,2.0,0.50,0.74,0.47
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444,0.090909,0.181818,0.751880,Matchweek 1,0.218,0.782,6,14,2,4,3,10,3,2,2,5,2,1,2,8,375,918,9,28,7,7,12,9,39,8,38,35,13,13,17/18,0.61,0.90,8.0,0.25,27.0,0.15,0.61,0.90,49.0,0.37,52.0,0.65,0.0,0.0,3.0,0.67,0.80,0.60
2,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,14.7143,0.166667,0.238095,0.617284,Matchweek 1,0.456,0.544,9,14,4,5,3,3,2,1,4,8,0,1,3,1,595,682,11,24,2,2,11,11,28,22,60,64,22,25,17/18,0.70,0.74,16.0,0.13,14.0,0.29,0.70,0.74,84.0,0.42,71.0,0.44,2.0,0.5,2.0,0.00,0.63,0.75
3,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,12.0000,0.416667,0.303030,0.303030,Matchweek 1,0.287,0.713,16,9,6,2,8,2,2,5,9,2,3,1,5,4,419,775,19,19,6,6,14,5,26,17,34,52,12,18,17/18,0.65,0.86,16.0,0.38,24.0,0.21,0.65,0.86,76.0,0.34,68.0,0.37,2.0,1.0,1.0,0.00,0.79,0.81
4,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,12.8824,0.617284,0.250000,0.153846,Matchweek 1,0.596,0.404,29,4,2,0,13,0,0,2,16,2,NaN,NaN,13,1,707,532,38,7,15,13,12,8,11,38,47,42,17,18,17/18,0.83,0.78,35.0,0.34,8.0,0.25,0.83,0.78,46.0,0.39,71.0,0.49,1.0,0.0,0.0,0.00,0.83,0.73


In [49]:
#fyllum nú upp í nan gildi með meðaltali liðs.
nan_cols = Epl_data_ready.columns[Epl_data_ready.isna().any()]

for col in nan_cols:
    Epl_data_ready[col] = pd.to_numeric(Epl_data_ready[col], errors='coerce')

home_cols = [col for col in nan_cols if col.startswith("Home_")]
away_cols = [col for col in nan_cols if col.startswith("Away_")]

for col in home_cols:
    Epl_data_ready[col] = Epl_data_ready.groupby("Home_Team")[col].transform(lambda x: x.fillna(x.mean()))

for col in away_cols:
    Epl_data_ready[col] = Epl_data_ready.groupby("Away_Team")[col].transform(lambda x: x.fillna(x.mean()))

In [56]:
nan_rows = Epl_data_ready[Epl_data_ready.isnull().T.any()]
nan_rows
#tómt svo getunm byrjað á feature engineering

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,OddsH,OddsD,OddsA,Matchweek,Home_Possession,Away_Possession,Home_Total_Shots,Away_Total_Shots,Home_Shots_On_Target,Away_Shots_On_Target,Home_Corners,Away_Corners,Home_Saves,Away_Saves,Home_Shots_Off_Target,Away_Shots_Off_Target,Home_Shots_Inside_the_Box,Away_Shots_Inside_the_Box,Home_Shots_Outside_the_Box,Away_Shots_Outside_the_Box,Home_Touches,Away_Touches,Home_Touches_in_the_opposition_box,Away_Touches_in_the_opposition_box,Home_Blocks,Away_Blocks,Home_Interceptions,Away_Interceptions,Home_Clearances,Away_Clearances,Home_Duels_Won,Away_Duels_Won,Home_Aerial_Duels_Won,Away_Aerial_Duels_Won,Season,Home_Passes_Completed,Away_Passes_Completed,Home_Total_Crosses,Home_Total_Crosses_Completed,Away_Total_Crosses,Away_Total_Crosses_Completed,Home_Total_Passes_Completed,Away_Total_Passes_Completed,Home_Long_Passes,Home_Long_Passes_Completed,Away_Long_Passes,Away_Long_Passes_Completed,Home_Through_Balls,Home_Through_Balls_Completed,Away_Through_Balls,Away_Through_Balls_Completed,Home_Tackles_Won_Completed,Away_Tackles_Won_Completed


In [55]:
#vistum nú datasettið sem er tilbúið
pick_insert = open('../data/Epl_data_ready.pickle','wb')
pickle.dump(Epl_data_ready, pick_insert)
pick_insert.close()

In [ ]:
#lesum inn ef þarf
pick_read = open('../data/Epl_data_ready.pickle','rb')
Epl_data_ready = pickle.load(pick_read)
pick_read.close()


In [57]:
#hvaða lið vann? Heima, draw eða útilið, aðeins öðruvísi en svipað fall að ofan.
def match_winner(winner):
  winner['target'] = 'W'
  winner['target'][winner['Home_Goals']==winner['Away_Goals']]='D'
  winner['target'][winner['Home_Goals']<winner['Away_Goals']]='L'

  return winner



In [58]:
Epl_data_ready = match_winner(Epl_data_ready)

In [59]:
Epl_data_ready.head()

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,OddsH,OddsD,OddsA,Matchweek,Home_Possession,Away_Possession,Home_Total_Shots,Away_Total_Shots,Home_Shots_On_Target,Away_Shots_On_Target,Home_Corners,Away_Corners,Home_Saves,Away_Saves,Home_Shots_Off_Target,Away_Shots_Off_Target,Home_Shots_Inside_the_Box,Away_Shots_Inside_the_Box,Home_Shots_Outside_the_Box,Away_Shots_Outside_the_Box,Home_Touches,Away_Touches,Home_Touches_in_the_opposition_box,Away_Touches_in_the_opposition_box,Home_Blocks,Away_Blocks,Home_Interceptions,Away_Interceptions,Home_Clearances,Away_Clearances,Home_Duels_Won,Away_Duels_Won,Home_Aerial_Duels_Won,Away_Aerial_Duels_Won,Season,Home_Passes_Completed,Away_Passes_Completed,Home_Total_Crosses,Home_Total_Crosses_Completed,Away_Total_Crosses,Away_Total_Crosses_Completed,Home_Total_Passes_Completed,Away_Total_Passes_Completed,Home_Long_Passes,Home_Long_Passes_Completed,Away_Long_Passes,Away_Long_Passes_Completed,Home_Through_Balls,Home_Through_Balls_Completed,Away_Through_Balls,Away_Through_Balls_Completed,Home_Tackles_Won_Completed,Away_Tackles_Won_Completed,target
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455,0.653595,0.222222,0.153846,Matchweek 1,0.700,0.300,27.0,6.0,10.0,3.0,9.0,4.0,0.0,6.0,9.0,3.0,3.000000,0.000000,12.0,1.0,859.0,459.0,30.0,11.0,9.0,8.0,13.0,11.0,30.0,24.0,69.0,48.0,18.0,18.0,17/18,0.85,0.63,20.0,0.30,18.0,0.22,0.85,0.63,55.0,0.67,70.0,0.46,2.0,0.5,2.0,0.50,0.74,0.47,W
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444,0.090909,0.181818,0.751880,Matchweek 1,0.218,0.782,6.0,14.0,2.0,4.0,3.0,10.0,3.0,2.0,2.0,5.0,2.000000,1.000000,2.0,8.0,375.0,918.0,9.0,28.0,7.0,7.0,12.0,9.0,39.0,8.0,38.0,35.0,13.0,13.0,17/18,0.61,0.90,8.0,0.25,27.0,0.15,0.61,0.90,49.0,0.37,52.0,0.65,0.0,0.0,3.0,0.67,0.80,0.60,L
2,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,14.7143,0.166667,0.238095,0.617284,Matchweek 1,0.456,0.544,9.0,14.0,4.0,5.0,3.0,3.0,2.0,1.0,4.0,8.0,0.000000,1.000000,3.0,1.0,595.0,682.0,11.0,24.0,2.0,2.0,11.0,11.0,28.0,22.0,60.0,64.0,22.0,25.0,17/18,0.70,0.74,16.0,0.13,14.0,0.29,0.70,0.74,84.0,0.42,71.0,0.44,2.0,0.5,2.0,0.00,0.63,0.75,D
3,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,12.0000,0.416667,0.303030,0.303030,Matchweek 1,0.287,0.713,16.0,9.0,6.0,2.0,8.0,2.0,2.0,5.0,9.0,2.0,3.000000,1.000000,5.0,4.0,419.0,775.0,19.0,19.0,6.0,6.0,14.0,5.0,26.0,17.0,34.0,52.0,12.0,18.0,17/18,0.65,0.86,16.0,0.38,24.0,0.21,0.65,0.86,76.0,0.34,68.0,0.37,2.0,1.0,1.0,0.00,0.79,0.81,W
4,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,12.8824,0.617284,0.250000,0.153846,Matchweek 1,0.596,0.404,29.0,4.0,2.0,0.0,13.0,0.0,0.0,2.0,16.0,2.0,1.953125,0.833333,13.0,1.0,707.0,532.0,38.0,7.0,15.0,13.0,12.0,8.0,11.0,38.0,47.0,42.0,17.0,18.0,17/18,0.83,0.78,35.0,0.34,8.0,0.25,0.83,0.78,46.0,0.39,71.0,0.49,1.0,0.0,0.0,0.00,0.83,0.73,D


In [ ]:
def assign_gameweeks(df):
    df = df.sort_values(['Season', 'Date']).reset_index(drop=True)

    #setjum upp gw, 10 leikir per viku 380/10
    df['GW'] = df.groupby('Season').cumcount().floordiv(10) + 1

    return df


In [69]:
Epl_data_ready = assign_gameweeks(Epl_data_ready)

In [ ]:
#setjum inn stig, úrslit og heildar stig á hverjum tíma fyrir leikinn í röðinni.
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

def get_match_result(stat_season):
    teams = {team: [] for team in stat_season['Home_Team'].unique()}
    
    for _, row in stat_season.iterrows():
        if row['target'] == 'W':
            teams[row['Home_Team']].append('W')
            teams[row['Away_Team']].append('L')
        elif row['target'] == 'L':
            teams[row['Home_Team']].append('L')
            teams[row['Away_Team']].append('W')
        else:
            teams[row['Home_Team']].append('D')
            teams[row['Away_Team']].append('D')

    gw_range = range(1, int(stat_season['GW'].max()) + 1)
    return pd.DataFrame(teams, index=gw_range).T

def get_cumulative_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2, matchres_points.shape[1] + 1):
        matchres_points[i] = matchres_points[i] + matchres_points[i - 1]
    matchres_points.insert(0, 0, [0] * matchres.shape[0])  # pre-GW 1
    return matchres_points

def get_agg_points(df):
    HTP_all = []
    ATP_all = []
    
    for season in df['Season'].unique():
        stat_season = df[df['Season'] == season].copy()
        stat_season.sort_values(by='GW', inplace=True)

        matchres = get_match_result(stat_season)
        cum_pts = get_cumulative_points(matchres)

        HTP = []
        ATP = []
        match_counters = {team: 0 for team in matchres.index}

        for _, row in stat_season.iterrows():
            ht = row['Home_Team']
            at = row['Away_Team']

            HTP.append(cum_pts.loc[ht][match_counters[ht]])
            ATP.append(cum_pts.loc[at][match_counters[at]])

            match_counters[ht] += 1
            match_counters[at] += 1

        HTP_all.extend(HTP)
        ATP_all.extend(ATP)

    df['HTP'] = HTP_all
    df['ATP'] = ATP_all
    return df


In [88]:
Epl_data_ready= get_agg_points(Epl_data_ready)

In [ ]:
import pandas as pd
import numpy as np

#Hér setjum við upp föll sem reikna form og breyta því í stig á rúllanndi 5 leikjum
def get_match_result(stat):
    teams = {team: [] for team in pd.concat([stat['Home_Team'], stat['Away_Team']]).unique()}

    for _, row in stat.iterrows():
        home, away, result = row['Home_Team'], row['Away_Team'], row['target']
        if result == 'W':
            teams[home].append('W')
            teams[away].append('L')
        elif result == 'L':
            teams[away].append('W')
            teams[home].append('L')
        else:
            teams[home].append('D')
            teams[away].append('D')

    max_gw = stat['GW'].max()
    return pd.DataFrame(data=teams, index=range(1, max_gw + 1)).T

#Skráum form sem WDL
def get_form(form_df, num_games):
    form_final = pd.DataFrame(index=form_df.index)
    for gw in range(1, form_df.shape[1] + 1):
        start = max(1, gw - num_games)
        form_final[gw] = form_df.loc[:, start:gw - 1].apply(lambda x: ''.join(x.dropna()), axis=1)
    return form_final

def get_points(char):
    return {'W': 3, 'D': 1, 'L': 0}.get(char, 0)

def get_form_points(form_str):
    if pd.isna(form_str) or form_str == "":
        return 0
    return sum(get_points(c) for c in form_str)

#Form fyrir tímabilið
def add_form_for_season(season_df, num_games=5):
    season_df = season_df.sort_values('GW').reset_index(drop=True)
    form_df = get_match_result(season_df)
    form_str_df = get_form(form_df, num_games)

    home_form_str, away_form_str, home_pts, away_pts = [], [], [], []

    for _, row in season_df.iterrows():
        gw = row['GW']
        home = row['Home_Team']
        away = row['Away_Team']

        hf = form_str_df.at[home, gw]
        af = form_str_df.at[away, gw]
        home_form_str.append(hf)
        away_form_str.append(af)
        home_pts.append(get_form_points(hf))
        away_pts.append(get_form_points(af))

    season_df[f'HM{num_games}'] = home_form_str
    season_df[f'AM{num_games}'] = away_form_str
    season_df[f'HTFormPts'] = home_pts
    season_df[f'ATFormPts'] = away_pts

    return season_df

def add_form_all_seasons(df, num_games=5):
    return pd.concat([
        add_form_for_season(season_df, num_games)
        for _, season_df in df.groupby('Season')
    ]).reset_index(drop=True)


In [139]:
Epl_data_ready = add_form_all_seasons(Epl_data_ready, num_games=5)

In [140]:
Epl_data_ready.tail()

,GameId,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,OddsH,OddsD,OddsA,Matchweek,Home_Possession,Away_Possession,Home_Total_Shots,Away_Total_Shots,Home_Shots_On_Target,Away_Shots_On_Target,Home_Corners,Away_Corners,Home_Saves,Away_Saves,Home_Shots_Off_Target,Away_Shots_Off_Target,Home_Shots_Inside_the_Box,Away_Shots_Inside_the_Box,Home_Shots_Outside_the_Box,Away_Shots_Outside_the_Box,Home_Touches,Away_Touches,Home_Touches_in_the_opposition_box,Away_Touches_in_the_opposition_box,Home_Blocks,Away_Blocks,Home_Interceptions,Away_Interceptions,Home_Clearances,Away_Clearances,Home_Duels_Won,Away_Duels_Won,Home_Aerial_Duels_Won,Away_Aerial_Duels_Won,Season,Home_Passes_Completed,Away_Passes_Completed,Home_Total_Crosses,Home_Total_Crosses_Completed,Away_Total_Crosses,Away_Total_Crosses_Completed,Home_Total_Passes_Completed,Away_Total_Passes_Completed,Home_Long_Passes,Home_Long_Passes_Completed,Away_Long_Passes,Away_Long_Passes_Completed,Home_Through_Balls,Home_Through_Balls_Completed,Away_Through_Balls,Away_Through_Balls_Completed,Home_Tackles_Won_Completed,Away_Tackles_Won_Completed,target,GW,HTP,ATP,HM5,AM5,HTFormPts,ATFormPts
3035,3036,2025/05/25,Newcastle United,Everton,0,1,1.1591,1.07279,6,1,6.2500,17.8667,0.751880,0.181818,0.117647,Matchweek 38,0.653,0.347,17.0,14.0,6.0,6.0,12.0,3.0,5.0,6.0,7.0,6.0,3.0,2.0,7.0,5.0,708.0,458.0,29.0,13.0,7.0,6.0,5.0,14.0,21.0,25.0,43.0,49.0,18.0,16.0,24/25,0.88,0.78,34.0,0.29,11.0,0.27,0.88,0.78,38.0,0.47,61.0,0.43,1.0,0.0,0.0,0.0,0.73,0.62,L,38,66,45,LWDWL,LLDWW,7,7
3036,3037,2025/05/25,Nottingham Forest,Chelsea,0,1,1.58014,1.30874,3,1,11.3889,14.1176,0.333333,0.266667,0.454545,Matchweek 38,0.518,0.482,10.0,6.0,2.0,2.0,7.0,4.0,1.0,2.0,4.0,2.0,1.0,1.0,2.0,2.0,607.0,577.0,28.0,11.0,7.0,5.0,3.0,4.0,22.0,36.0,37.0,38.0,10.0,9.0,24/25,0.84,0.80,33.0,0.21,11.0,0.09,0.84,0.80,49.0,0.43,51.0,0.25,2.0,0.5,0.0,0.0,0.64,0.67,L,38,65,66,WLDDW,WWWLW,8,12
3037,3038,2025/05/25,Fulham,Manchester City,0,2,2.21797,2.45923,5,16,22.1538,13.5455,0.200000,0.238095,0.625000,Matchweek 38,0.468,0.532,13.0,20.0,3.0,5.0,1.0,6.0,3.0,3.0,6.0,7.0,3.0,2.0,2.0,4.0,627.0,679.0,32.0,46.0,12.0,11.0,11.0,13.0,29.0,14.0,41.0,42.0,12.0,10.0,24/25,0.85,0.86,12.0,0.42,18.0,0.17,0.85,0.86,59.0,0.54,60.0,0.60,2.0,0.5,0.0,0.0,0.44,0.64,L,38,54,68,LWLLW,WWWDW,6,13
3038,3039,2025/05/25,Manchester United,Aston Villa,2,0,2.96259,0.335222,24,5,5.1071,26.9091,0.230947,0.238095,0.588235,Matchweek 38,0.668,0.332,26.0,6.0,10.0,1.0,4.0,3.0,1.0,8.0,14.0,2.0,6.0,1.0,11.0,2.0,781.0,445.0,55.0,18.0,5.0,5.0,7.0,6.0,19.0,20.0,52.0,40.0,22.0,11.0,24/25,0.88,0.72,19.0,0.32,9.0,0.00,0.88,0.72,47.0,0.66,60.0,0.33,5.0,0.6,2.0,0.0,0.75,0.56,W,38,39,66,LDLLL,WLWWW,1,12
3039,3040,2025/05/25,Bournemouth,Leicester,2,0,2.72449,0.156702,16,3,4.6471,9.2800,0.751880,0.190476,0.117647,Matchweek 38,0.627,0.373,20.0,3.0,7.0,0.0,6.0,1.0,0.0,5.0,7.0,3.0,2.0,0.0,10.0,1.0,681.0,482.0,52.0,12.0,9.0,6.0,7.0,6.0,11.0,30.0,62.0,55.0,22.0,13.0,24/25,0.82,0.74,22.0,0.32,8.0,0.25,0.82,0.74,48.0,0.33,51.0,0.29,1.0,1.0,0.0,0.0,0.60,0.67,W,38,53,25,DDWLL,LLWDW,5,7


In [141]:
#gameid
Epl_data_ready.insert(0, 'GameId', range(1, len(Epl_data_ready)+1))

ValueError: cannot insert GameId, already exists

In [142]:
Epl_data_ready

,GameId,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,OddsH,OddsD,OddsA,Matchweek,Home_Possession,Away_Possession,Home_Total_Shots,Away_Total_Shots,Home_Shots_On_Target,Away_Shots_On_Target,Home_Corners,Away_Corners,Home_Saves,Away_Saves,Home_Shots_Off_Target,Away_Shots_Off_Target,Home_Shots_Inside_the_Box,Away_Shots_Inside_the_Box,Home_Shots_Outside_the_Box,Away_Shots_Outside_the_Box,Home_Touches,Away_Touches,Home_Touches_in_the_opposition_box,Away_Touches_in_the_opposition_box,Home_Blocks,Away_Blocks,Home_Interceptions,Away_Interceptions,Home_Clearances,Away_Clearances,Home_Duels_Won,Away_Duels_Won,Home_Aerial_Duels_Won,Away_Aerial_Duels_Won,Season,Home_Passes_Completed,Away_Passes_Completed,Home_Total_Crosses,Home_Total_Crosses_Completed,Away_Total_Crosses,Away_Total_Crosses_Completed,Home_Total_Passes_Completed,Away_Total_Passes_Completed,Home_Long_Passes,Home_Long_Passes_Completed,Away_Long_Passes,Away_Long_Passes_Completed,Home_Through_Balls,Home_Through_Balls_Completed,Away_Through_Balls,Away_Through_Balls_Completed,Home_Tackles_Won_Completed,Away_Tackles_Won_Completed,target,GW,HTP,ATP,HM5,AM5,HTFormPts,ATFormPts
0,1,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455,0.653595,0.222222,0.153846,Matchweek 1,0.700,0.300,27.0,6.0,10.0,3.0,9.0,4.0,0.0,6.0,9.0,3.0,3.000000,0.000000,12.0,1.0,859.0,459.0,30.0,11.0,9.0,8.0,13.0,11.0,30.0,24.0,69.0,48.0,18.0,18.0,17/18,0.85,0.63,20.0,0.30,18.0,0.22,0.85,0.63,55.0,0.67,70.0,0.46,2.0,0.5,2.0,0.50,0.74,0.47,W,1,0,0,,,0,0
1,2,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444,0.090909,0.181818,0.751880,Matchweek 1,0.218,0.782,6.0,14.0,2.0,4.0,3.0,10.0,3.0,2.0,2.0,5.0,2.000000,1.000000,2.0,8.0,375.0,918.0,9.0,28.0,7.0,7.0,12.0,9.0,39.0,8.0,38.0,35.0,13.0,13.0,17/18,0.61,0.90,8.0,0.25,27.0,0.15,0.61,0.90,49.0,0.37,52.0,0.65,0.0,0.0,3.0,0.67,0.80,0.60,L,1,0,0,,,0,0
2,3,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,14.7143,0.166667,0.238095,0.617284,Matchweek 1,0.456,0.544,9.0,14.0,4.0,5.0,3.0,3.0,2.0,1.0,4.0,8.0,0.000000,1.000000,3.0,1.0,595.0,682.0,11.0,24.0,2.0,2.0,11.0,11.0,28.0,22.0,60.0,64.0,22.0,25.0,17/18,0.70,0.74,16.0,0.13,14.0,0.29,0.70,0.74,84.0,0.42,71.0,0.44,2.0,0.5,2.0,0.00,0.63,0.75,D,1,0,0,,,0,0
3,4,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,12.0000,0.416667,0.303030,0.303030,Matchweek 1,0.287,0.713,16.0,9.0,6.0,2.0,8.0,2.0,2.0,5.0,9.0,2.0,3.000000,1.000000,5.0,4.0,419.0,775.0,19.0,19.0,6.0,6.0,14.0,5.0,26.0,17.0,34.0,52.0,12.0,18.0,17/18,0.65,0.86,16.0,0.38,24.0,0.21,0.65,0.86,76.0,0.34,68.0,0.37,2.0,1.0,1.0,0.00,0.79,0.81,W,1,0,0,,,0,0
4,5,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,12.8824,0.617284,0.250000,0.153846,Matchweek 1,0.596,0.404,29.0,4.0,2.0,0.0,13.0,0.0,0.0,2.0,16.0,2.0,1.953125,0.833333,13.0,1.0,707.0,532.0,38.0,7.0,15.0,13.0,12.0,8.0,11.0,38.0,47.0,42.0,17.0,18.0,17/18,0.83,0.78,35.0,0.34,8.0,0.25,0.83,0.78,46.0,0.39,71.0,0.49,1.0,0.0,0.0,0.00,0.83,0.73,D,1,0,0,,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,3036,2025/05/25,Newcastle United,Everton,0,1,1.1591,1.07279,6,1,6.2500,17.8667,0.751880,0.181818,0.117647,Matchweek 38,0.653,0.347,17.0,14.0,6.0,6.0,12.0,3.0,5.0,6.0,7.0,6.0,3.000000,2.000000,7.0,5.0,708.0,458.0,29.0,13.0,7.0,6.0,5.0,14.0,21.0,25.0,43.0,49.0,18.0,16.0,24/25,0.88,0.78,34.0,0.29,11.0,0.27,0.88,0.78,38.0,0.47,61.0,0.43,1.0,0.0,0.0,0.00,0.73,0.62,L,38,66,45,LWDWL,LLDWW,7,7
3036,3037,2025/05/25,Nottingham Forest,Chelsea,0,1,1.58014,1.30874,3,1,11.3889,14.1176,0.333333,0.266667,0.454545,Matchweek 38,0.518,0.482,10.0,6.0,2.0,2.0,7.0,4.0,1.0,2.0,4.0,2.0,1.000000,1.000000,2.0,2.0,607.0,577.0,28.0,11.0,7.0,5.0,3.0,4.0,22

Hér vil ég gera smá breytingar á framsetningu. Laga til breytur með því að setja fram "rolling average" eða exponentially weighted gildi á þeim. Form breyta, hversu vel er lið að standa sig í seinustu 5 eða 10 leikjum. Þarf svo að færa til staðsetningu breytanna þar sem módelið má ekki spá fyrir(y) um leik x með tölfræði úr leik x. Get notað shift functionið í pandas eftir að ég hef búið til breyturnar. Einnig væri áhugavert að setja fram einhverskonar breytu sem mælir styrk liðs varnar og sóknarlega.

In [143]:
Away_cols = ['GameId'] + [col for col in Epl_data_ready.columns if 'Away' in col] + ['ATFormPts']
Away_df = Epl_data_ready[Away_cols]


In [ ]:
Home_cols = ['GameId'] + [col for col in Epl_data_ready.columns if 'Home' in col] + ['HTFormPts']
Home_df = Epl_data_ready[Home_cols]

In [148]:
Home_df.columns = [col.replace('Home_', '') for col in Home_df.columns]
Home_df.columns = [col.replace('HT', '') for col in Home_df.columns]

Away_df.columns = [col.replace('Away_', '') for col in Away_df.columns]
Away_df.columns = [col.replace('AT', '') for col in Away_df.columns]


In [150]:
Home_df['Venue'] = 'Home'
Away_df['Venue'] = 'Away'

In [221]:
stat_df = pd.concat([Home_df,Away_df]).sort_values(by='GameId').reset_index(drop=True)
stat_df

,GameId,Team,Goals,xG,Deep,ppda,Possession,Total_Shots,Shots_On_Target,Corners,Saves,Shots_Off_Target,Shots_Inside_the_Box,Shots_Outside_the_Box,Touches,Touches_in_the_opposition_box,Blocks,Interceptions,Clearances,Duels_Won,Aerial_Duels_Won,Passes_Completed,Total_Crosses,Total_Crosses_Completed,Total_Passes_Completed,Long_Passes,Long_Passes_Completed,Through_Balls,Through_Balls_Completed,Tackles_Won_Completed,FormPts,Venue
0,1,Arsenal,4,2.54329,13,5.4444,0.700,27.0,10.0,9.0,0.0,9.0,3.0,12.0,859.0,30.0,9.0,13.0,30.0,69.0,18.0,0.85,20.0,0.30,0.85,55.0,0.67,2.0,0.50,0.74,0,Home
1,1,Leicester,3,1.46495,2,13.5455,0.300,6.0,3.0,4.0,6.0,3.0,0.0,1.0,459.0,11.0,8.0,11.0,24.0,48.0,18.0,0.63,18.0,0.22,0.63,70.0,0.46,2.0,0.50,0.47,0,Away
2,2,Manchester City,2,1.86751,18,5.4444,0.782,14.0,4.0,10.0,2.0,5.0,1.0,8.0,918.0,28.0,7.0,9.0,8.0,35.0,13.0,0.90,27.0,0.15,0.90,52.0,0.65,3.0,0.67,0.60,0,Away
3,2,Brighton,0,0.276343,2,67.3333,0.218,6.0,2.0,3.0,3.0,2.0,2.0,2.0,375.0,9.0,7.0,12.0,39.0,38.0,13.0,0.61,8.0,0.25,0.61,49.0,0.37,0.0,0.00,0.80,0,Home
4,3,Liverpool,3,2.61549,10,14.7143,0.544,14.0,5.0,3.0,1.0,8.0,1.0,1.0,682.0,24.0,2.0,11.0,22.0,64.0,25.0,0.74,14.0,0.29,0.74,71.0,0.44,2.0,0.00,0.75,0,Away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,3038,Manchester City,2,2.45923,16,13.5455,0.532,20.0,5.0,6.0,3.0,7.0,2.0,4.0,679.0,46.0,11.0,13.0,14.0,42.0,10.0,0.86,18.0,0.17,0.86,60.0,0.60,0.0,0.00,0.64,13,Away
6076,3039,Manchester United,2,2.96259,24,5.1071,0.668,26.0,10.0,4.0,1.0,14.0,6.0,11.0,781.0,55.0,5.0,7.0,19.0,52.0,22.0,0.88,19.0,0.32,0.88,47.0,0.66,5.0,0.60,0.75,1,Home
6077,3039,Aston Villa,0,0.335222,5,26.9091,0.332,6.0,1.0,3.0,8.0,2.0,1.0,2.0,445.0,18.0,5.0,6.0,20.0,40.0,11.0,0.72,9.0,0.00,0.72,60.0,0.33,2.0,0.00,0.56,12,Away
6078,3040,Leicester,0,0.156702,3,9.2800,0.373,3.0,0.0,1.0,5.0,3.0,0.0,1.0,482.0,12.0,6.0,6.0,30.0,55.0,13.0,0.74,8.0,0.25,0.74,51.0,0.29,0.0,0.00,0.67,7,Away


In [222]:
stat_cols = stat_df.drop(['GameId', 'Team', 'FormPts', 'Venue'], axis=1)
#droppa string/texta dálkum
for s in stat_cols:
    stat_df[s+'_ema5'] = stat_df.groupby('Team')[s].transform(lambda x:x.ewm(span=5).mean()) # reyna að ná form styrk
    stat_df[s+'_ema38'] = stat_df.groupby('Team')[s].transform(lambda x:x.ewm(span=38).mean()) # reyna að ná tímabilsstyrk
    stat_df[s+'_5ma'] = stat_df.groupby('Team')[s].transform(lambda x:x.rolling(window=5, min_periods=1).mean()) #mean form
stat_df.tail()

,GameId,Team,Goals,xG,Deep,ppda,Possession,Total_Shots,Shots_On_Target,Corners,Saves,Shots_Off_Target,Shots_Inside_the_Box,Shots_Outside_the_Box,Touches,Touches_in_the_opposition_box,Blocks,Interceptions,Clearances,Duels_Won,Aerial_Duels_Won,Passes_Completed,Total_Crosses,Total_Crosses_Completed,Total_Passes_Completed,Long_Passes,Long_Passes_Completed,Through_Balls,Through_Balls_Completed,Tackles_Won_Completed,FormPts,Venue,Goals_ema5,Goals_ema38,Goals_5ma,xG_ema5,xG_ema38,xG_5ma,Deep_ema5,Deep_ema38,Deep_5ma,ppda_ema5,ppda_ema38,ppda_5ma,Possession_ema5,Possession_ema38,Possession_5ma,Total_Shots_ema5,Total_Shots_ema38,Total_Shots_5ma,Shots_On_Target_ema5,Shots_On_Target_ema38,Shots_On_Target_5ma,Corners_ema5,Corners_ema38,Corners_5ma,Saves_ema5,Saves_ema38,Saves_5ma,Shots_Off_Target_ema5,Shots_Off_Target_ema38,Shots_Off_Target_5ma,Shots_Inside_the_Box_ema5,Shots_Inside_the_Box_ema38,Shots_Inside_the_Box_5ma,Shots_Outside_the_Box_ema5,Shots_Outside_the_Box_ema38,Shots_Outside_the_Box_5ma,Touches_ema5,Touches_ema38,Touches_5ma,Touches_in_the_opposition_box_ema5,Touches_in_the_opposition_box_ema38,Touches_in_the_opposition_box_5ma,Blocks_ema5,Blocks_ema38,Blocks_5ma,Interceptions_ema5,Interceptions_ema38,Interceptions_5ma,Clearances_ema5,Clearances_ema38,Clearances_5ma,Duels_Won_ema5,Duels_Won_ema38,Duels_Won_5ma,Aerial_Duels_Won_ema5,Aerial_Duels_Won_ema38,Aerial_Duels_Won_5ma,Passes_Completed_ema5,Passes_Completed_ema38,Passes_Completed_5ma,Total_Crosses_ema5,Total_Crosses_ema38,Total_Crosses_5ma,Total_Crosses_Completed_ema5,Total_Crosses_Completed_ema38,Total_Crosses_Completed_5ma,Total_Passes_Completed_ema5,Total_Passes_Completed_ema38,Total_Passes_Completed_5ma,Long_Passes_ema5,Long_Passes_ema38,Long_Passes_5ma,Long_Passes_Completed_ema5,Long_Passes_Completed_ema38,Long_Passes_Completed_5ma,Through_Balls_ema5,Through_Balls_ema38,Through_Balls_5ma,Through_Balls_Completed_ema5,Through_Balls_Completed_ema38,Through_Balls_Completed_5ma,Tackles_Won_Completed_ema5,Tackles_Won_Completed_ema38,Tackles_Won_Completed_5ma
6075,3038,Manchester City,2,2.45923,16,13.5455,0.532,20.0,5.0,6.0,3.0,7.0,2.0,4.0,679.0,46.0,11.0,13.0,14.0,42.0,10.0,0.86,18.0,0.17,0.86,60.0,0.60,0.0,0.0,0.64,13,Away,1.860659,1.975095,1.6,1.883923,1.945312,1.572687,13.169472,11.186562,12.4,13.482369,12.695734,12.84706,0.600673,0.620595,0.6156,16.873847,15.778542,16.2,4.952002,5.717093,4.6,6.415803,6.207552,7.6,1.584116,2.078180,1.2,6.120463,5.513275,6.2,1.914479,2.388730,1.8,5.969511,5.699075,6.2,755.575215,783.263255,758.0,40.572934,36.788222,41.6,8.569743,7.194690,8.0,7.950370,6.133638,6.6,14.863855,15.263655,13.8,39.909485,41.389622,40.0,8.950698,8.472122,8.8,0.885050,0.895416,0.892,18.876602,17.015089,21.8,0.222995,0.232735,0.252,0.885050,0.895416,0.892,45.696087,38.237349,42.6,0.573094,0.569235,0.576,1.575977,2.144216,1.8,0.362744,0.389878,0.400,0.544656,0.586770,0.526
6076,3039,Manchester United,2,2.96259,24,5.1071,0.668,26.0,10.0,4.0,1.0,14.0,6.0,11.0,781.0,55.0,5.0,7.0,19.0,52.0,22.0,0.88,19.0,0.32,0.88,47.0,0.66,5.0,0.6,0.75,1,Home,1.109537,1.200266,1.2,1.795856,1.493436,1.731504,13.139146,8.794243,12.0,8.759444,10.634478,9.24334,0.582013,0.534554,0.5748,17.237079,14.499351,17.8,5.664576,4.704537,5.4,4.713425,5.287953,5.4,1.694893,2.739827,1.6,8.205313,5.151969,7.2,3.759029,2.709046,3.6,6.789213,5.377224,6.6,706.838023,679.872225,692.6,34.951741,28.473806,34.2,6.720882,8.428894,8.8,5.661707,8.394312,5.4,18.318410,20.249587,19.4,50.096229,50.480233,47.0,15.834853,13.427727,13.8,0.853626,0.836548,0.848,18.208243,16.946539,18.8,0.243494,0.242771,0.226,0.853626,0.836548,0.848,52.981278,50.076550,52.6,0.556942,0.468134,0.518,2.915014,2.496798,3.2,0.395090,0.364585,0.400,0.647021,0.610927,0.644
6077,3039,Aston Villa,0,0.335222,5,26.9091,0.332,6.0,1.0,3.0,8.0,2.0,1.0,2.0,445.0,18.0,5.0,6.0,20.0,40.0,11.0,0.72,9.0,0.00,0.72,60.0,0.33,2.0,0.0,0.56,12,Away,1.130791,1.573759,1.0,1.229491,1.664642,1.259756,7.888647,7.546090,6.6,17.495695,14.115071,17.22078,0.456431,

In [ ]:
#shift by one til að nota ekki núverandi leik sem part af spágögnum, droppum öllum non ema eða 5ma gildum
feature_cols = ['GameId', 'Team', 'FormPts', 'Venue'] + [col for col in stat_df.columns if any(x in col for x in ['_ema5', '_ema38', '_5ma'])]
stat_df_features = stat_df[feature_cols].copy()

home_df = stat_df_features[stat_df_features['Venue'] == 'Home'].copy()
away_df = stat_df_features[stat_df_features['Venue'] == 'Away'].copy()

ignore_cols = ['GameId', 'Team', 'FormPts', 'Venue']
cols_to_shift = [col for col in feature_cols if col not in ignore_cols]

home_df[cols_to_shift] = home_df.groupby('Team')[cols_to_shift].shift(1)
away_df[cols_to_shift] = away_df.groupby('Team')[cols_to_shift].shift(1)

home_df_renamed = home_df.add_prefix('Home_')
away_df_renamed = away_df.add_prefix('Away_')

home_df_renamed['GameId'] = home_df['GameId']
away_df_renamed['GameId'] = away_df['GameId']

stat_df_ready = pd.merge(home_df_renamed, away_df_renamed, on='GameId')
stat_df_ready


,Home_GameId,Home_Team,Home_FormPts,Home_Venue,Home_Goals_ema5,Home_Goals_ema38,Home_Goals_5ma,Home_xG_ema5,Home_xG_ema38,Home_xG_5ma,Home_Deep_ema5,Home_Deep_ema38,Home_Deep_5ma,Home_ppda_ema5,Home_ppda_ema38,Home_ppda_5ma,Home_Possession_ema5,Home_Possession_ema38,Home_Possession_5ma,Home_Total_Shots_ema5,Home_Total_Shots_ema38,Home_Total_Shots_5ma,Home_Shots_On_Target_ema5,Home_Shots_On_Target_ema38,Home_Shots_On_Target_5ma,Home_Corners_ema5,Home_Corners_ema38,Home_Corners_5ma,Home_Saves_ema5,Home_Saves_ema38,Home_Saves_5ma,Home_Shots_Off_Target_ema5,Home_Shots_Off_Target_ema38,Home_Shots_Off_Target_5ma,Home_Shots_Inside_the_Box_ema5,Home_Shots_Inside_the_Box_ema38,Home_Shots_Inside_the_Box_5ma,Home_Shots_Outside_the_Box_ema5,Home_Shots_Outside_the_Box_ema38,Home_Shots_Outside_the_Box_5ma,Home_Touches_ema5,Home_Touches_ema38,Home_Touches_5ma,Home_Touches_in_the_opposition_box_ema5,Home_Touches_in_the_opposition_box_ema38,Home_Touches_in_the_opposition_box_5ma,Home_Blocks_ema5,Home_Blocks_ema38,Home_Blocks_5ma,Home_Interceptions_ema5,Home_Interceptions_ema38,Home_Interceptions_5ma,Home_Clearances_ema5,Home_Clearances_ema38,Home_Clearances_5ma,Home_Duels_Won_ema5,Home_Duels_Won_ema38,Home_Duels_Won_5ma,Home_Aerial_Duels_Won_ema5,Home_Aerial_Duels_Won_ema38,Home_Aerial_Duels_Won_5ma,Home_Passes_Completed_ema5,Home_Passes_Completed_ema38,Home_Passes_Completed_5ma,Home_Total_Crosses_ema5,Home_Total_Crosses_ema38,Home_Total_Crosses_5ma,Home_Total_Crosses_Completed_ema5,Home_Total_Crosses_Completed_ema38,Home_Total_Crosses_Completed_5ma,Home_Total_Passes_Completed_ema5,Home_Total_Passes_Completed_ema38,Home_Total_Passes_Completed_5ma,Home_Long_Passes_ema5,Home_Long_Passes_ema38,Home_Long_Passes_5ma,Home_Long_Passes_Completed_ema5,Home_Long_Passes_Completed_ema38,Home_Long_Passes_Completed_5ma,Home_Through_Balls_ema5,Home_Through_Balls_ema38,Home_Through_Balls_5ma,Home_Through_Balls_Completed_ema5,Home_Through_Balls_Completed_ema38,Home_Through_Balls_Completed_5ma,Home_Tackles_Won_Completed_ema5,Home_Tackles_Won_Completed_ema38,Home_Tackles_Won_Completed_5ma,GameId,Away_GameId,Away_Team,Away_FormPts,Away_Venue,Away_Goals_ema5,Away_Goals_ema38,Away_Goals_5ma,Away_xG_ema5,Away_xG_ema38,Away_xG_5ma,Away_Deep_ema5,Away_Deep_ema38,Away_Deep_5ma,Away_ppda_ema5,Away_ppda_ema38,Away_ppda_5ma,Away_Possession_ema5,Away_Possession_ema38,Away_Possession_5ma,Away_Total_Shots_ema5,Away_Total_Shots_ema38,Away_Total_Shots_5ma,Away_Shots_On_Target_ema5,Away_Shots_On_Target_ema38,Away_Shots_On_Target_5ma,Away_Corners_ema5,Away_Corners_ema38,Away_Corners_5ma,Away_Saves_ema5,Away_Saves_ema38,Away_Saves_5ma,Away_Shots_Off_Target_ema5,Away_Shots_Off_Target_ema38,Away_Shots_Off_Target_5ma,Away_Shots_Inside_the_Box_ema5,Away_Shots_Inside_the_Box_ema38,Away_Shots_Inside_the_Box_5ma,Away_Shots_Outside_the_Box_ema5,Away_Shots_Outside_the_Box_ema38,Away_Shots_Outside_the_Box_5ma,Away_Touches_ema5,Away_Touches_ema38,Away_Touches_5ma,Away_Touches_in_the_opposition_box_ema5,Away_Touches_in_the_opposition_box_ema38,Away_Touches_in_the_opposition_box_5ma,Away_Blocks_ema5,Away_Blocks_ema38,Away_Blocks_5ma,Away_Interceptions_ema5,Away_Interceptions_ema38,Away_Interceptions_5ma,Away_Clearances_ema5,Away_Clearances_ema38,Away_Clearances_5ma,Away_Duels_Won_ema5,Away_Duels_Won_ema38,Away_Duels_Won_5ma,Away_Aerial_Duels_Won_ema5,Away_Aerial_Duels_Won_ema38,Away_Aerial_Duels_Won_5ma,Away_Passes_Completed_ema5,Away_Passes_Completed_ema38,Away_Passes_Completed_5ma,Away_Total_Crosses_ema5,Away_Total_Crosses_ema38,Away_Total_Crosses_5ma,Away_Total_Crosses_Completed_ema5,Away_Total_Crosses_Completed_ema38,Away_Total_Crosses_Completed_5ma,Away_Total_Passes_Completed_ema5,Away_Total_Passes_Completed_ema38,Away_Total_Passes_Completed_5ma,Away_Long_Passes_ema5,Away_Long_Passes_ema38,Away_Long_Passes_5ma,Away_Long_Passes_Completed_ema5,Away_Long_Passes_Completed_ema38,Away_Long_Passes_Completed_5ma,Away_Through_Balls_ema5,Away_Through_Balls_ema38,Away_Through_Balls_5ma,Away_Through

In [ ]:
#þetta veldur nokkum nan breytum, bæði fyrstu leikirnir sökum shift og svo koma upp nan þegar að nýliðar komast upp í deild þeirra bestu
#annað hvort væri hægt að gefa þeim eitthvað global mean eða bara droppa þeim röðum, ath að þegar
# spáð er þarf að gefa þeim gildi fyrst...
#spurning er hversu mikið af þessu ég þarf.

In [ ]:
first_cols = ['GameId', 'Home_Team', 'Away_Team', 'Home_FormPts', 'Away_FormPts']

remaining_cols = [col for col in stat_df_ready.columns if col not in first_cols]

stat_df_ready = stat_df_ready[first_cols + remaining_cols]
stat_df_ready = stat_df_ready.drop(['Home_GameId', 'Away_GameId', 'Home_Venue', 'Away_Venue'], axis=1)

stat_df_ready

,GameId,Home_Team,Away_Team,Home_FormPts,Away_FormPts,Home_Goals_ema5,Home_Goals_ema38,Home_Goals_5ma,Home_xG_ema5,Home_xG_ema38,Home_xG_5ma,Home_Deep_ema5,Home_Deep_ema38,Home_Deep_5ma,Home_ppda_ema5,Home_ppda_ema38,Home_ppda_5ma,Home_Possession_ema5,Home_Possession_ema38,Home_Possession_5ma,Home_Total_Shots_ema5,Home_Total_Shots_ema38,Home_Total_Shots_5ma,Home_Shots_On_Target_ema5,Home_Shots_On_Target_ema38,Home_Shots_On_Target_5ma,Home_Corners_ema5,Home_Corners_ema38,Home_Corners_5ma,Home_Saves_ema5,Home_Saves_ema38,Home_Saves_5ma,Home_Shots_Off_Target_ema5,Home_Shots_Off_Target_ema38,Home_Shots_Off_Target_5ma,Home_Shots_Inside_the_Box_ema5,Home_Shots_Inside_the_Box_ema38,Home_Shots_Inside_the_Box_5ma,Home_Shots_Outside_the_Box_ema5,Home_Shots_Outside_the_Box_ema38,Home_Shots_Outside_the_Box_5ma,Home_Touches_ema5,Home_Touches_ema38,Home_Touches_5ma,Home_Touches_in_the_opposition_box_ema5,Home_Touches_in_the_opposition_box_ema38,Home_Touches_in_the_opposition_box_5ma,Home_Blocks_ema5,Home_Blocks_ema38,Home_Blocks_5ma,Home_Interceptions_ema5,Home_Interceptions_ema38,Home_Interceptions_5ma,Home_Clearances_ema5,Home_Clearances_ema38,Home_Clearances_5ma,Home_Duels_Won_ema5,Home_Duels_Won_ema38,Home_Duels_Won_5ma,Home_Aerial_Duels_Won_ema5,Home_Aerial_Duels_Won_ema38,Home_Aerial_Duels_Won_5ma,Home_Passes_Completed_ema5,Home_Passes_Completed_ema38,Home_Passes_Completed_5ma,Home_Total_Crosses_ema5,Home_Total_Crosses_ema38,Home_Total_Crosses_5ma,Home_Total_Crosses_Completed_ema5,Home_Total_Crosses_Completed_ema38,Home_Total_Crosses_Completed_5ma,Home_Total_Passes_Completed_ema5,Home_Total_Passes_Completed_ema38,Home_Total_Passes_Completed_5ma,Home_Long_Passes_ema5,Home_Long_Passes_ema38,Home_Long_Passes_5ma,Home_Long_Passes_Completed_ema5,Home_Long_Passes_Completed_ema38,Home_Long_Passes_Completed_5ma,Home_Through_Balls_ema5,Home_Through_Balls_ema38,Home_Through_Balls_5ma,Home_Through_Balls_Completed_ema5,Home_Through_Balls_Completed_ema38,Home_Through_Balls_Completed_5ma,Home_Tackles_Won_Completed_ema5,Home_Tackles_Won_Completed_ema38,Home_Tackles_Won_Completed_5ma,Away_Goals_ema5,Away_Goals_ema38,Away_Goals_5ma,Away_xG_ema5,Away_xG_ema38,Away_xG_5ma,Away_Deep_ema5,Away_Deep_ema38,Away_Deep_5ma,Away_ppda_ema5,Away_ppda_ema38,Away_ppda_5ma,Away_Possession_ema5,Away_Possession_ema38,Away_Possession_5ma,Away_Total_Shots_ema5,Away_Total_Shots_ema38,Away_Total_Shots_5ma,Away_Shots_On_Target_ema5,Away_Shots_On_Target_ema38,Away_Shots_On_Target_5ma,Away_Corners_ema5,Away_Corners_ema38,Away_Corners_5ma,Away_Saves_ema5,Away_Saves_ema38,Away_Saves_5ma,Away_Shots_Off_Target_ema5,Away_Shots_Off_Target_ema38,Away_Shots_Off_Target_5ma,Away_Shots_Inside_the_Box_ema5,Away_Shots_Inside_the_Box_ema38,Away_Shots_Inside_the_Box_5ma,Away_Shots_Outside_the_Box_ema5,Away_Shots_Outside_the_Box_ema38,Away_Shots_Outside_the_Box_5ma,Away_Touches_ema5,Away_Touches_ema38,Away_Touches_5ma,Away_Touches_in_the_opposition_box_ema5,Away_Touches_in_the_opposition_box_ema38,Away_Touches_in_the_opposition_box_5ma,Away_Blocks_ema5,Away_Blocks_ema38,Away_Blocks_5ma,Away_Interceptions_ema5,Away_Interceptions_ema38,Away_Interceptions_5ma,Away_Clearances_ema5,Away_Clearances_ema38,Away_Clearances_5ma,Away_Duels_Won_ema5,Away_Duels_Won_ema38,Away_Duels_Won_5ma,Away_Aerial_Duels_Won_ema5,Away_Aerial_Duels_Won_ema38,Away_Aerial_Duels_Won_5ma,Away_Passes_Completed_ema5,Away_Passes_Completed_ema38,Away_Passes_Completed_5ma,Away_Total_Crosses_ema5,Away_Total_Crosses_ema38,Away_Total_Crosses_5ma,Away_Total_Crosses_Completed_ema5,Away_Total_Crosses_Completed_ema38,Away_Total_Crosses_Completed_5ma,Away_Total_Passes_Completed_ema5,Away_Total_Passes_Completed_ema38,Away_Total_Passes_Completed_5ma,Away_Long_Passes_ema5,Away_Long_Passes_ema38,Away_Long_Passes_5ma,Away_Long_Passes_Completed_ema5,Away_Long_Passes_Completed_ema38,Away_Long_Passes_Completed_5ma,Away_Through_Balls_ema5,Away_Through_Balls_ema38,Away_Through_Balls_5ma,Away_Through_Balls_Completed_ema5,Away_Through_Balls_Compl

In [272]:
#bæta við relegated teams fyrir nan og early season indicator fyrir fyrstu 5 leiki hvers tímabils
#reikna út hvaða lið féllu og gefa nýliða þau gildi til að byrja með.
#bæta við ranking, elo/glicko
#bæta við nokkrum dálkum fyrir líkön, form, streaks, styrkur
#notum gögn úr epl_data_ready
Epl_data_ready['EarlySeason'] = Epl_data_ready.groupby('Season')['GW'].transform(lambda x: x <= 5)
Epl_data_ready['EarlySeason'] = Epl_data_ready['EarlySeason'].astype(int)

In [ ]:
#glicko
from glicko2 import Player
import pandas as pd

#skrá lið og úthluta "Player"
teams = pd.concat([Epl_data_ready['Home_Team'], Epl_data_ready['Away_Team']]).unique()
glicko_players = {team: Player() for team in teams}

#listi til að geyma ratings
rating_records = []

for i, row in Epl_data_ready.sort_values(['Season', 'GW']).iterrows():
    home = row['Home_Team']
    away = row['Away_Team']
    result = row['target']  
    p_home = glicko_players[home]
    p_away = glicko_players[away]

    #Glicko úrslit
    if result == 'W':
        home_score = 1.0
    elif result == 'D':
        home_score = 0.5
    else:
        home_score = 0.0

    p_home.update_player([p_away.getRating()], [p_away.getRd()], [home_score])
    p_away.update_player([p_home.getRating()], [p_home.getRd()], [1 - home_score])

    rating_records.append({
        'GameId': row['GameId'],
        'Home_Team': home,
        'Away_Team': away,
        'Home_Rating': p_home.getRating(),
        'Away_Rating': p_away.getRating(),
        'Season': row['Season'],
        'GW': row['GW']
    })

rating_df = pd.DataFrame(rating_records)
#shifta til að nýta ekki núverandi leik í spár
rating_df['Home_Rating'] = rating_df.groupby('Home_Team')['Home_Rating'].shift(1)
rating_df['Away_Rating'] = rating_df.groupby('Away_Team')['Away_Rating'].shift(1)



In [ ]:
stat_df_ready = pd.merge(stat_df_ready, rating_df[['GameId', 'Home_Rating', 'Away_Rating']], on='GameId', how='left')
stat_df_ready

,GameId,Home_Team,Away_Team,Home_FormPts,Away_FormPts,Home_Goals_ema5,Home_Goals_ema38,Home_Goals_5ma,Home_xG_ema5,Home_xG_ema38,Home_xG_5ma,Home_Deep_ema5,Home_Deep_ema38,Home_Deep_5ma,Home_ppda_ema5,Home_ppda_ema38,Home_ppda_5ma,Home_Possession_ema5,Home_Possession_ema38,Home_Possession_5ma,Home_Total_Shots_ema5,Home_Total_Shots_ema38,Home_Total_Shots_5ma,Home_Shots_On_Target_ema5,Home_Shots_On_Target_ema38,Home_Shots_On_Target_5ma,Home_Corners_ema5,Home_Corners_ema38,Home_Corners_5ma,Home_Saves_ema5,Home_Saves_ema38,Home_Saves_5ma,Home_Shots_Off_Target_ema5,Home_Shots_Off_Target_ema38,Home_Shots_Off_Target_5ma,Home_Shots_Inside_the_Box_ema5,Home_Shots_Inside_the_Box_ema38,Home_Shots_Inside_the_Box_5ma,Home_Shots_Outside_the_Box_ema5,Home_Shots_Outside_the_Box_ema38,Home_Shots_Outside_the_Box_5ma,Home_Touches_ema5,Home_Touches_ema38,Home_Touches_5ma,Home_Touches_in_the_opposition_box_ema5,Home_Touches_in_the_opposition_box_ema38,Home_Touches_in_the_opposition_box_5ma,Home_Blocks_ema5,Home_Blocks_ema38,Home_Blocks_5ma,Home_Interceptions_ema5,Home_Interceptions_ema38,Home_Interceptions_5ma,Home_Clearances_ema5,Home_Clearances_ema38,Home_Clearances_5ma,Home_Duels_Won_ema5,Home_Duels_Won_ema38,Home_Duels_Won_5ma,Home_Aerial_Duels_Won_ema5,Home_Aerial_Duels_Won_ema38,Home_Aerial_Duels_Won_5ma,Home_Passes_Completed_ema5,Home_Passes_Completed_ema38,Home_Passes_Completed_5ma,Home_Total_Crosses_ema5,Home_Total_Crosses_ema38,Home_Total_Crosses_5ma,Home_Total_Crosses_Completed_ema5,Home_Total_Crosses_Completed_ema38,Home_Total_Crosses_Completed_5ma,Home_Total_Passes_Completed_ema5,Home_Total_Passes_Completed_ema38,Home_Total_Passes_Completed_5ma,Home_Long_Passes_ema5,Home_Long_Passes_ema38,Home_Long_Passes_5ma,Home_Long_Passes_Completed_ema5,Home_Long_Passes_Completed_ema38,Home_Long_Passes_Completed_5ma,Home_Through_Balls_ema5,Home_Through_Balls_ema38,Home_Through_Balls_5ma,Home_Through_Balls_Completed_ema5,Home_Through_Balls_Completed_ema38,Home_Through_Balls_Completed_5ma,Home_Tackles_Won_Completed_ema5,Home_Tackles_Won_Completed_ema38,Home_Tackles_Won_Completed_5ma,Away_Goals_ema5,Away_Goals_ema38,Away_Goals_5ma,Away_xG_ema5,Away_xG_ema38,Away_xG_5ma,Away_Deep_ema5,Away_Deep_ema38,Away_Deep_5ma,Away_ppda_ema5,Away_ppda_ema38,Away_ppda_5ma,Away_Possession_ema5,Away_Possession_ema38,Away_Possession_5ma,Away_Total_Shots_ema5,Away_Total_Shots_ema38,Away_Total_Shots_5ma,Away_Shots_On_Target_ema5,Away_Shots_On_Target_ema38,Away_Shots_On_Target_5ma,Away_Corners_ema5,Away_Corners_ema38,Away_Corners_5ma,Away_Saves_ema5,Away_Saves_ema38,Away_Saves_5ma,Away_Shots_Off_Target_ema5,Away_Shots_Off_Target_ema38,Away_Shots_Off_Target_5ma,Away_Shots_Inside_the_Box_ema5,Away_Shots_Inside_the_Box_ema38,Away_Shots_Inside_the_Box_5ma,Away_Shots_Outside_the_Box_ema5,Away_Shots_Outside_the_Box_ema38,Away_Shots_Outside_the_Box_5ma,Away_Touches_ema5,Away_Touches_ema38,Away_Touches_5ma,Away_Touches_in_the_opposition_box_ema5,Away_Touches_in_the_opposition_box_ema38,Away_Touches_in_the_opposition_box_5ma,Away_Blocks_ema5,Away_Blocks_ema38,Away_Blocks_5ma,Away_Interceptions_ema5,Away_Interceptions_ema38,Away_Interceptions_5ma,Away_Clearances_ema5,Away_Clearances_ema38,Away_Clearances_5ma,Away_Duels_Won_ema5,Away_Duels_Won_ema38,Away_Duels_Won_5ma,Away_Aerial_Duels_Won_ema5,Away_Aerial_Duels_Won_ema38,Away_Aerial_Duels_Won_5ma,Away_Passes_Completed_ema5,Away_Passes_Completed_ema38,Away_Passes_Completed_5ma,Away_Total_Crosses_ema5,Away_Total_Crosses_ema38,Away_Total_Crosses_5ma,Away_Total_Crosses_Completed_ema5,Away_Total_Crosses_Completed_ema38,Away_Total_Crosses_Completed_5ma,Away_Total_Passes_Completed_ema5,Away_Total_Passes_Completed_ema38,Away_Total_Passes_Completed_5ma,Away_Long_Passes_ema5,Away_Long_Passes_ema38,Away_Long_Passes_5ma,Away_Long_Passes_Completed_ema5,Away_Long_Passes_Completed_ema38,Away_Long_Passes_Completed_5ma,Away_Through_Balls_ema5,Away_Through_Balls_ema38,Away_Through_Balls_5ma,Away_Through_Balls_Completed_ema5,Away_Through_Balls_Compl

In [ ]:
#Finnum hvaða lið féllu 17/18 til að gefa liðum sem koma upp 
final_gw_df = Epl_data_ready[(Epl_data_ready['Season'] == '17/18') & (Epl_data_ready['GW'] == 38)]

points_data = pd.concat([
    final_gw_df[['Home_Team', 'HTP']].rename(columns={'Home_Team': 'Team', 'HTP': 'Points'}),
    final_gw_df[['Away_Team', 'ATP']].rename(columns={'Away_Team': 'Team', 'ATP': 'Points'})
])

points_data = points_data.drop_duplicates(subset='Team')

relegated_teams = points_data.sort_values('Points').head(3)



,Team,Points
374,Stoke,30
376,West Bromwich Albion,31
374,Swansea,33


In [304]:
#fá average frá þeim til að gefa liðum sem koma upp í PL
relegated_team_names = relegated_teams['Team'].tolist()

relegated_stats_home = stat_df_ready[stat_df_ready['Home_Team'].isin(relegated_team_names)]
relegated_stats_away = stat_df_ready[stat_df_ready['Away_Team'].isin(relegated_team_names)]

relegated_stats_home = relegated_stats_home.dropna()
relegated_stats_away = relegated_stats_away.dropna()

template_stats_home = relegated_stats_home.select_dtypes(include='number').mean()
template_stats_away = relegated_stats_home.select_dtypes(include='number').mean()



In [305]:
template_stats_home = template_stats_home.drop(['GameId', 'Home_FormPts', 'Away_FormPts'])
template_stats_away = template_stats_away.drop(['GameId', 'Home_FormPts', 'Away_FormPts'])
#getum nú sett þetta inn í stað nan þegar df er tilbúinn.

In [ ]:
#bætum nú við því seinasta... Form, heima og away, hvað er langt síðan heimalið tapaði eða útilið tapaði og win streak heima og away
import pandas as pd

def get_streak_length(results_list, target_results):

    streak = 0
    for r in reversed(results_list):
        if r in target_results:
            streak += 1
        else:
            break
    return streak

def build_streak_features(df):
    teams = pd.concat([df['Home_Team'], df['Away_Team']]).unique()
    
    #setja upp streaks fyrir öll lið
    team_results = {team: {'all': [], 'home': [], 'away': []} for team in teams}
    
    #listi fyrir streaks
    home_win_streak_all = []
    home_loss_streak_all = []
    home_unbeaten_streak_all = []
    home_win_streak_home = []
    home_loss_streak_home = []
    home_unbeaten_streak_home = []
    away_win_streak_all = []
    away_loss_streak_all = []
    away_unbeaten_streak_all = []
    away_win_streak_away = []
    away_loss_streak_away = []
    away_unbeaten_streak_away = []
    
    df = df.sort_values(['Season', 'GW']).reset_index(drop=True)
    current_season = None
    
    for idx, row in df.iterrows():
        #Endurræsa á nýju tímabili
        if row['Season'] != current_season:
            current_season = row['Season']
            team_results = {team: {'all': [], 'home': [], 'away': []} for team in teams}
        
        home = row['Home_Team']
        away = row['Away_Team']
        result = row['target']
        
        #Skráum streaks fyrir núverandi leik
        
        #Heimalið streaks
        home_all = team_results[home]['all']
        home_home = team_results[home]['home']
        
        home_win_streak_all.append(get_streak_length(home_all, {'W'}))
        home_loss_streak_all.append(get_streak_length(home_all, {'L'}))
        home_unbeaten_streak_all.append(get_streak_length(home_all, {'W', 'D'}))
        
        home_win_streak_home.append(get_streak_length(home_home, {'W'}))
        home_loss_streak_home.append(get_streak_length(home_home, {'L'}))
        home_unbeaten_streak_home.append(get_streak_length(home_home, {'W', 'D'}))
        
        #útilið streaks
        away_all = team_results[away]['all']
        away_away = team_results[away]['away']
        
        away_win_streak_all.append(get_streak_length(away_all, {'W'}))
        away_loss_streak_all.append(get_streak_length(away_all, {'L'}))
        away_unbeaten_streak_all.append(get_streak_length(away_all, {'W', 'D'}))
        
        away_win_streak_away.append(get_streak_length(away_away, {'W'}))
        away_loss_streak_away.append(get_streak_length(away_away, {'L'}))
        away_unbeaten_streak_away.append(get_streak_length(away_away, {'W', 'D'}))
        
        
        #úrslit heimaliðs
        team_results[home]['all'].append(result)
        team_results[home]['home'].append(result)
        
        #úrslit útiliðs
        if result == 'W':
            away_result = 'L'
        elif result == 'L':
            away_result = 'W'
        else:
            away_result = 'D'
        
        team_results[away]['all'].append(away_result)
        team_results[away]['away'].append(away_result)
    
    #Bæta við streaks
    df['Home_WinStreak_All'] = home_win_streak_all
    df['Home_LossStreak_All'] = home_loss_streak_all
    df['Home_UnbeatenStreak_All'] = home_unbeaten_streak_all
    df['Home_WinStreak_Home'] = home_win_streak_home
    df['Home_LossStreak_Home'] = home_loss_streak_home
    df['Home_UnbeatenStreak_Home'] = home_unbeaten_streak_home
    
    df['Away_WinStreak_All'] = away_win_streak_all
    df['Away_LossStreak_All'] = away_loss_streak_all
    df['Away_UnbeatenStreak_All'] = away_unbeaten_streak_all
    df['Away_WinStreak_Away'] = away_win_streak_away
    df['Away_LossStreak_Away'] = away_loss_streak_away
    df['Away_UnbeatenStreak_Away'] = away_unbeaten_streak_away
    
    return df


In [310]:
Epl_data_ready = build_streak_features(Epl_data_ready)

In [312]:
Epl_data_lysandi = Epl_data_ready.filter(
    ['GameId', 'Home_Team', 'Away_Team', 'Home_Goals', 'Away_Goals', 'GW'] +
    [col for col in Epl_data_ready.columns if any(x in col for x in ['Streak'])],
    axis=1
)

In [321]:
df_ready = pd.merge(left=Epl_data_lysandi, right=stat_df_ready,
                  left_on= ['GameId','Home_Team', 'Away_Team'],
                   right_on=['GameId','Home_Team', 'Away_Team'],
                  how='left')

In [ ]:
#nú þarf bara að henda út fyrstu leikjum tímabilsins þar sem engin gögn 
#voru til staðar og filla fyrstu leiki hjá nýliðum með meðalgildum liða sem féllu
first_40_cleaned = df_ready.iloc[:40].dropna()
rest = df_ready.iloc[40:]
df_ready = pd.concat([first_40_cleaned, rest], ignore_index=True)
df_ready

,GameId,Home_Team,Away_Team,Home_Goals,Away_Goals,GW,Home_WinStreak_All,Home_LossStreak_All,Home_UnbeatenStreak_All,Home_WinStreak_Home,Home_LossStreak_Home,Home_UnbeatenStreak_Home,Away_WinStreak_All,Away_LossStreak_All,Away_UnbeatenStreak_All,Away_WinStreak_Away,Away_LossStreak_Away,Away_UnbeatenStreak_Away,Home_FormPts,Away_FormPts,Home_Goals_ema5,Home_Goals_ema38,Home_Goals_5ma,Home_xG_ema5,Home_xG_ema38,Home_xG_5ma,Home_Deep_ema5,Home_Deep_ema38,Home_Deep_5ma,Home_ppda_ema5,Home_ppda_ema38,Home_ppda_5ma,Home_Possession_ema5,Home_Possession_ema38,Home_Possession_5ma,Home_Total_Shots_ema5,Home_Total_Shots_ema38,Home_Total_Shots_5ma,Home_Shots_On_Target_ema5,Home_Shots_On_Target_ema38,Home_Shots_On_Target_5ma,Home_Corners_ema5,Home_Corners_ema38,Home_Corners_5ma,Home_Saves_ema5,Home_Saves_ema38,Home_Saves_5ma,Home_Shots_Off_Target_ema5,Home_Shots_Off_Target_ema38,Home_Shots_Off_Target_5ma,Home_Shots_Inside_the_Box_ema5,Home_Shots_Inside_the_Box_ema38,Home_Shots_Inside_the_Box_5ma,Home_Shots_Outside_the_Box_ema5,Home_Shots_Outside_the_Box_ema38,Home_Shots_Outside_the_Box_5ma,Home_Touches_ema5,Home_Touches_ema38,Home_Touches_5ma,Home_Touches_in_the_opposition_box_ema5,Home_Touches_in_the_opposition_box_ema38,Home_Touches_in_the_opposition_box_5ma,Home_Blocks_ema5,Home_Blocks_ema38,Home_Blocks_5ma,Home_Interceptions_ema5,Home_Interceptions_ema38,Home_Interceptions_5ma,Home_Clearances_ema5,Home_Clearances_ema38,Home_Clearances_5ma,Home_Duels_Won_ema5,Home_Duels_Won_ema38,Home_Duels_Won_5ma,Home_Aerial_Duels_Won_ema5,Home_Aerial_Duels_Won_ema38,Home_Aerial_Duels_Won_5ma,Home_Passes_Completed_ema5,Home_Passes_Completed_ema38,Home_Passes_Completed_5ma,Home_Total_Crosses_ema5,Home_Total_Crosses_ema38,Home_Total_Crosses_5ma,Home_Total_Crosses_Completed_ema5,Home_Total_Crosses_Completed_ema38,Home_Total_Crosses_Completed_5ma,Home_Total_Passes_Completed_ema5,Home_Total_Passes_Completed_ema38,Home_Total_Passes_Completed_5ma,Home_Long_Passes_ema5,Home_Long_Passes_ema38,Home_Long_Passes_5ma,Home_Long_Passes_Completed_ema5,Home_Long_Passes_Completed_ema38,Home_Long_Passes_Completed_5ma,Home_Through_Balls_ema5,Home_Through_Balls_ema38,Home_Through_Balls_5ma,Home_Through_Balls_Completed_ema5,Home_Through_Balls_Completed_ema38,Home_Through_Balls_Completed_5ma,Home_Tackles_Won_Completed_ema5,Home_Tackles_Won_Completed_ema38,Home_Tackles_Won_Completed_5ma,Away_Goals_ema5,Away_Goals_ema38,Away_Goals_5ma,Away_xG_ema5,Away_xG_ema38,Away_xG_5ma,Away_Deep_ema5,Away_Deep_ema38,Away_Deep_5ma,Away_ppda_ema5,Away_ppda_ema38,Away_ppda_5ma,Away_Possession_ema5,Away_Possession_ema38,Away_Possession_5ma,Away_Total_Shots_ema5,Away_Total_Shots_ema38,Away_Total_Shots_5ma,Away_Shots_On_Target_ema5,Away_Shots_On_Target_ema38,Away_Shots_On_Target_5ma,Away_Corners_ema5,Away_Corners_ema38,Away_Corners_5ma,Away_Saves_ema5,Away_Saves_ema38,Away_Saves_5ma,Away_Shots_Off_Target_ema5,Away_Shots_Off_Target_ema38,Away_Shots_Off_Target_5ma,Away_Shots_Inside_the_Box_ema5,Away_Shots_Inside_the_Box_ema38,Away_Shots_Inside_the_Box_5ma,Away_Shots_Outside_the_Box_ema5,Away_Shots_Outside_the_Box_ema38,Away_Shots_Outside_the_Box_5ma,Away_Touches_ema5,Away_Touches_ema38,Away_Touches_5ma,Away_Touches_in_the_opposition_box_ema5,Away_Touches_in_the_opposition_box_ema38,Away_Touches_in_the_opposition_box_5ma,Away_Blocks_ema5,Away_Blocks_ema38,Away_Blocks_5ma,Away_Interceptions_ema5,Away_Interceptions_ema38,Away_Interceptions_5ma,Away_Clearances_ema5,Away_Clearances_ema38,Away_Clearances_5ma,Away_Duels_Won_ema5,Away_Duels_Won_ema38,Away_Duels_Won_5ma,Away_Aerial_Duels_Won_ema5,Away_Aerial_Duels_Won_ema38,Away_Aerial_Duels_Won_5ma,Away_Passes_Completed_ema5,Away_Passes_Completed_ema38,Away_Passes_Completed_5ma,Away_Total_Crosses_ema5,Away_Total_Crosses_ema38,Away_Total_Crosses_5ma,Away_Total_Crosses_Completed_ema5,Away_Total_Crosses_Completed_ema38,Away_Total_Crosses_Completed_5ma,Away_Total_Passes_Completed_ema5,Away_Total_Passes_Completed_ema38,Away_Total_Passes_Completed_5ma,Away_Long

In [327]:
#athugum hvaða leikir eru með nan
nan_rows = df_ready[df_ready.isnull().any(axis=1)]
nan_rows


,GameId,Home_Team,Away_Team,Home_Goals,Away_Goals,GW,Home_WinStreak_All,Home_LossStreak_All,Home_UnbeatenStreak_All,Home_WinStreak_Home,Home_LossStreak_Home,Home_UnbeatenStreak_Home,Away_WinStreak_All,Away_LossStreak_All,Away_UnbeatenStreak_All,Away_WinStreak_Away,Away_LossStreak_Away,Away_UnbeatenStreak_Away,Home_FormPts,Away_FormPts,Home_Goals_ema5,Home_Goals_ema38,Home_Goals_5ma,Home_xG_ema5,Home_xG_ema38,Home_xG_5ma,Home_Deep_ema5,Home_Deep_ema38,Home_Deep_5ma,Home_ppda_ema5,Home_ppda_ema38,Home_ppda_5ma,Home_Possession_ema5,Home_Possession_ema38,Home_Possession_5ma,Home_Total_Shots_ema5,Home_Total_Shots_ema38,Home_Total_Shots_5ma,Home_Shots_On_Target_ema5,Home_Shots_On_Target_ema38,Home_Shots_On_Target_5ma,Home_Corners_ema5,Home_Corners_ema38,Home_Corners_5ma,Home_Saves_ema5,Home_Saves_ema38,Home_Saves_5ma,Home_Shots_Off_Target_ema5,Home_Shots_Off_Target_ema38,Home_Shots_Off_Target_5ma,Home_Shots_Inside_the_Box_ema5,Home_Shots_Inside_the_Box_ema38,Home_Shots_Inside_the_Box_5ma,Home_Shots_Outside_the_Box_ema5,Home_Shots_Outside_the_Box_ema38,Home_Shots_Outside_the_Box_5ma,Home_Touches_ema5,Home_Touches_ema38,Home_Touches_5ma,Home_Touches_in_the_opposition_box_ema5,Home_Touches_in_the_opposition_box_ema38,Home_Touches_in_the_opposition_box_5ma,Home_Blocks_ema5,Home_Blocks_ema38,Home_Blocks_5ma,Home_Interceptions_ema5,Home_Interceptions_ema38,Home_Interceptions_5ma,Home_Clearances_ema5,Home_Clearances_ema38,Home_Clearances_5ma,Home_Duels_Won_ema5,Home_Duels_Won_ema38,Home_Duels_Won_5ma,Home_Aerial_Duels_Won_ema5,Home_Aerial_Duels_Won_ema38,Home_Aerial_Duels_Won_5ma,Home_Passes_Completed_ema5,Home_Passes_Completed_ema38,Home_Passes_Completed_5ma,Home_Total_Crosses_ema5,Home_Total_Crosses_ema38,Home_Total_Crosses_5ma,Home_Total_Crosses_Completed_ema5,Home_Total_Crosses_Completed_ema38,Home_Total_Crosses_Completed_5ma,Home_Total_Passes_Completed_ema5,Home_Total_Passes_Completed_ema38,Home_Total_Passes_Completed_5ma,Home_Long_Passes_ema5,Home_Long_Passes_ema38,Home_Long_Passes_5ma,Home_Long_Passes_Completed_ema5,Home_Long_Passes_Completed_ema38,Home_Long_Passes_Completed_5ma,Home_Through_Balls_ema5,Home_Through_Balls_ema38,Home_Through_Balls_5ma,Home_Through_Balls_Completed_ema5,Home_Through_Balls_Completed_ema38,Home_Through_Balls_Completed_5ma,Home_Tackles_Won_Completed_ema5,Home_Tackles_Won_Completed_ema38,Home_Tackles_Won_Completed_5ma,Away_Goals_ema5,Away_Goals_ema38,Away_Goals_5ma,Away_xG_ema5,Away_xG_ema38,Away_xG_5ma,Away_Deep_ema5,Away_Deep_ema38,Away_Deep_5ma,Away_ppda_ema5,Away_ppda_ema38,Away_ppda_5ma,Away_Possession_ema5,Away_Possession_ema38,Away_Possession_5ma,Away_Total_Shots_ema5,Away_Total_Shots_ema38,Away_Total_Shots_5ma,Away_Shots_On_Target_ema5,Away_Shots_On_Target_ema38,Away_Shots_On_Target_5ma,Away_Corners_ema5,Away_Corners_ema38,Away_Corners_5ma,Away_Saves_ema5,Away_Saves_ema38,Away_Saves_5ma,Away_Shots_Off_Target_ema5,Away_Shots_Off_Target_ema38,Away_Shots_Off_Target_5ma,Away_Shots_Inside_the_Box_ema5,Away_Shots_Inside_the_Box_ema38,Away_Shots_Inside_the_Box_5ma,Away_Shots_Outside_the_Box_ema5,Away_Shots_Outside_the_Box_ema38,Away_Shots_Outside_the_Box_5ma,Away_Touches_ema5,Away_Touches_ema38,Away_Touches_5ma,Away_Touches_in_the_opposition_box_ema5,Away_Touches_in_the_opposition_box_ema38,Away_Touches_in_the_opposition_box_5ma,Away_Blocks_ema5,Away_Blocks_ema38,Away_Blocks_5ma,Away_Interceptions_ema5,Away_Interceptions_ema38,Away_Interceptions_5ma,Away_Clearances_ema5,Away_Clearances_ema38,Away_Clearances_5ma,Away_Duels_Won_ema5,Away_Duels_Won_ema38,Away_Duels_Won_5ma,Away_Aerial_Duels_Won_ema5,Away_Aerial_Duels_Won_ema38,Away_Aerial_Duels_Won_5ma,Away_Passes_Completed_ema5,Away_Passes_Completed_ema38,Away_Passes_Completed_5ma,Away_Total_Crosses_ema5,Away_Total_Crosses_ema38,Away_Total_Crosses_5ma,Away_Total_Crosses_Completed_ema5,Away_Total_Crosses_Completed_ema38,Away_Total_Crosses_Completed_5ma,Away_Total_Passes_Completed_ema5,Away_Total_Passes_Completed_ema38,Away_Total_Passes_Completed_5ma,Away_Long

In [ ]:
#sjáum að þetta eru nýiðar svo við gefum þeim meðalgildi falliða

for col in [col for col in df_ready.columns if col.startswith('Home_')]:
    if col in template_stats_home.index:
        df_ready[col] = df_ready[col].fillna(template_stats_home[col])

for col in [col for col in df_ready.columns if col.startswith('Away_')]:
    if col in template_stats_away.index:
        df_ready[col] = df_ready[col].fillna(template_stats_away[col])




Filling NaNs in Home_Goals_ema5 with value 0.8430214902915649
Filling NaNs in Home_Goals_ema38 with value 0.8179927910415515
Filling NaNs in Home_Goals_5ma with value 0.8273972602739726
Filling NaNs in Home_xG_ema5 with value 0.9152999440051998
Filling NaNs in Home_xG_ema38 with value 0.8800099825336061
Filling NaNs in Home_xG_5ma with value 0.8959169397260274
Filling NaNs in Home_Deep_ema5 with value 4.171882012675881
Filling NaNs in Home_Deep_ema38 with value 3.9435103619326637
Filling NaNs in Home_Deep_5ma with value 4.100913242009132
Filling NaNs in Home_ppda_ema5 with value 16.79921952454859
Filling NaNs in Home_ppda_ema38 with value 16.718295657192268
Filling NaNs in Home_ppda_5ma with value 16.895615570776254
Filling NaNs in Home_Possession_ema5 with value 0.4090841042421901
Filling NaNs in Home_Possession_ema38 with value 0.4098983400073967
Filling NaNs in Home_Possession_5ma with value 0.4071210909539677
Filling NaNs in Home_Total_Shots_ema5 with value 9.5610655822794
Filling 

In [348]:
nan_rows = df_ready[df_ready.isnull().any(axis=1)]
nan_rows

,GameId,Home_Team,Away_Team,Home_Goals,Away_Goals,GW,Home_WinStreak_All,Home_LossStreak_All,Home_UnbeatenStreak_All,Home_WinStreak_Home,Home_LossStreak_Home,Home_UnbeatenStreak_Home,Away_WinStreak_All,Away_LossStreak_All,Away_UnbeatenStreak_All,Away_WinStreak_Away,Away_LossStreak_Away,Away_UnbeatenStreak_Away,Home_FormPts,Away_FormPts,Home_Goals_ema5,Home_Goals_ema38,Home_Goals_5ma,Home_xG_ema5,Home_xG_ema38,Home_xG_5ma,Home_Deep_ema5,Home_Deep_ema38,Home_Deep_5ma,Home_ppda_ema5,Home_ppda_ema38,Home_ppda_5ma,Home_Possession_ema5,Home_Possession_ema38,Home_Possession_5ma,Home_Total_Shots_ema5,Home_Total_Shots_ema38,Home_Total_Shots_5ma,Home_Shots_On_Target_ema5,Home_Shots_On_Target_ema38,Home_Shots_On_Target_5ma,Home_Corners_ema5,Home_Corners_ema38,Home_Corners_5ma,Home_Saves_ema5,Home_Saves_ema38,Home_Saves_5ma,Home_Shots_Off_Target_ema5,Home_Shots_Off_Target_ema38,Home_Shots_Off_Target_5ma,Home_Shots_Inside_the_Box_ema5,Home_Shots_Inside_the_Box_ema38,Home_Shots_Inside_the_Box_5ma,Home_Shots_Outside_the_Box_ema5,Home_Shots_Outside_the_Box_ema38,Home_Shots_Outside_the_Box_5ma,Home_Touches_ema5,Home_Touches_ema38,Home_Touches_5ma,Home_Touches_in_the_opposition_box_ema5,Home_Touches_in_the_opposition_box_ema38,Home_Touches_in_the_opposition_box_5ma,Home_Blocks_ema5,Home_Blocks_ema38,Home_Blocks_5ma,Home_Interceptions_ema5,Home_Interceptions_ema38,Home_Interceptions_5ma,Home_Clearances_ema5,Home_Clearances_ema38,Home_Clearances_5ma,Home_Duels_Won_ema5,Home_Duels_Won_ema38,Home_Duels_Won_5ma,Home_Aerial_Duels_Won_ema5,Home_Aerial_Duels_Won_ema38,Home_Aerial_Duels_Won_5ma,Home_Passes_Completed_ema5,Home_Passes_Completed_ema38,Home_Passes_Completed_5ma,Home_Total_Crosses_ema5,Home_Total_Crosses_ema38,Home_Total_Crosses_5ma,Home_Total_Crosses_Completed_ema5,Home_Total_Crosses_Completed_ema38,Home_Total_Crosses_Completed_5ma,Home_Total_Passes_Completed_ema5,Home_Total_Passes_Completed_ema38,Home_Total_Passes_Completed_5ma,Home_Long_Passes_ema5,Home_Long_Passes_ema38,Home_Long_Passes_5ma,Home_Long_Passes_Completed_ema5,Home_Long_Passes_Completed_ema38,Home_Long_Passes_Completed_5ma,Home_Through_Balls_ema5,Home_Through_Balls_ema38,Home_Through_Balls_5ma,Home_Through_Balls_Completed_ema5,Home_Through_Balls_Completed_ema38,Home_Through_Balls_Completed_5ma,Home_Tackles_Won_Completed_ema5,Home_Tackles_Won_Completed_ema38,Home_Tackles_Won_Completed_5ma,Away_Goals_ema5,Away_Goals_ema38,Away_Goals_5ma,Away_xG_ema5,Away_xG_ema38,Away_xG_5ma,Away_Deep_ema5,Away_Deep_ema38,Away_Deep_5ma,Away_ppda_ema5,Away_ppda_ema38,Away_ppda_5ma,Away_Possession_ema5,Away_Possession_ema38,Away_Possession_5ma,Away_Total_Shots_ema5,Away_Total_Shots_ema38,Away_Total_Shots_5ma,Away_Shots_On_Target_ema5,Away_Shots_On_Target_ema38,Away_Shots_On_Target_5ma,Away_Corners_ema5,Away_Corners_ema38,Away_Corners_5ma,Away_Saves_ema5,Away_Saves_ema38,Away_Saves_5ma,Away_Shots_Off_Target_ema5,Away_Shots_Off_Target_ema38,Away_Shots_Off_Target_5ma,Away_Shots_Inside_the_Box_ema5,Away_Shots_Inside_the_Box_ema38,Away_Shots_Inside_the_Box_5ma,Away_Shots_Outside_the_Box_ema5,Away_Shots_Outside_the_Box_ema38,Away_Shots_Outside_the_Box_5ma,Away_Touches_ema5,Away_Touches_ema38,Away_Touches_5ma,Away_Touches_in_the_opposition_box_ema5,Away_Touches_in_the_opposition_box_ema38,Away_Touches_in_the_opposition_box_5ma,Away_Blocks_ema5,Away_Blocks_ema38,Away_Blocks_5ma,Away_Interceptions_ema5,Away_Interceptions_ema38,Away_Interceptions_5ma,Away_Clearances_ema5,Away_Clearances_ema38,Away_Clearances_5ma,Away_Duels_Won_ema5,Away_Duels_Won_ema38,Away_Duels_Won_5ma,Away_Aerial_Duels_Won_ema5,Away_Aerial_Duels_Won_ema38,Away_Aerial_Duels_Won_5ma,Away_Passes_Completed_ema5,Away_Passes_Completed_ema38,Away_Passes_Completed_5ma,Away_Total_Crosses_ema5,Away_Total_Crosses_ema38,Away_Total_Crosses_5ma,Away_Total_Crosses_Completed_ema5,Away_Total_Crosses_Completed_ema38,Away_Total_Crosses_Completed_5ma,Away_Total_Passes_Completed_ema5,Away_Total_Passes_Completed_ema38,Away_Total_Passes_Completed_5ma,Away_Long

In [349]:
df_ready

,GameId,Home_Team,Away_Team,Home_Goals,Away_Goals,GW,Home_WinStreak_All,Home_LossStreak_All,Home_UnbeatenStreak_All,Home_WinStreak_Home,Home_LossStreak_Home,Home_UnbeatenStreak_Home,Away_WinStreak_All,Away_LossStreak_All,Away_UnbeatenStreak_All,Away_WinStreak_Away,Away_LossStreak_Away,Away_UnbeatenStreak_Away,Home_FormPts,Away_FormPts,Home_Goals_ema5,Home_Goals_ema38,Home_Goals_5ma,Home_xG_ema5,Home_xG_ema38,Home_xG_5ma,Home_Deep_ema5,Home_Deep_ema38,Home_Deep_5ma,Home_ppda_ema5,Home_ppda_ema38,Home_ppda_5ma,Home_Possession_ema5,Home_Possession_ema38,Home_Possession_5ma,Home_Total_Shots_ema5,Home_Total_Shots_ema38,Home_Total_Shots_5ma,Home_Shots_On_Target_ema5,Home_Shots_On_Target_ema38,Home_Shots_On_Target_5ma,Home_Corners_ema5,Home_Corners_ema38,Home_Corners_5ma,Home_Saves_ema5,Home_Saves_ema38,Home_Saves_5ma,Home_Shots_Off_Target_ema5,Home_Shots_Off_Target_ema38,Home_Shots_Off_Target_5ma,Home_Shots_Inside_the_Box_ema5,Home_Shots_Inside_the_Box_ema38,Home_Shots_Inside_the_Box_5ma,Home_Shots_Outside_the_Box_ema5,Home_Shots_Outside_the_Box_ema38,Home_Shots_Outside_the_Box_5ma,Home_Touches_ema5,Home_Touches_ema38,Home_Touches_5ma,Home_Touches_in_the_opposition_box_ema5,Home_Touches_in_the_opposition_box_ema38,Home_Touches_in_the_opposition_box_5ma,Home_Blocks_ema5,Home_Blocks_ema38,Home_Blocks_5ma,Home_Interceptions_ema5,Home_Interceptions_ema38,Home_Interceptions_5ma,Home_Clearances_ema5,Home_Clearances_ema38,Home_Clearances_5ma,Home_Duels_Won_ema5,Home_Duels_Won_ema38,Home_Duels_Won_5ma,Home_Aerial_Duels_Won_ema5,Home_Aerial_Duels_Won_ema38,Home_Aerial_Duels_Won_5ma,Home_Passes_Completed_ema5,Home_Passes_Completed_ema38,Home_Passes_Completed_5ma,Home_Total_Crosses_ema5,Home_Total_Crosses_ema38,Home_Total_Crosses_5ma,Home_Total_Crosses_Completed_ema5,Home_Total_Crosses_Completed_ema38,Home_Total_Crosses_Completed_5ma,Home_Total_Passes_Completed_ema5,Home_Total_Passes_Completed_ema38,Home_Total_Passes_Completed_5ma,Home_Long_Passes_ema5,Home_Long_Passes_ema38,Home_Long_Passes_5ma,Home_Long_Passes_Completed_ema5,Home_Long_Passes_Completed_ema38,Home_Long_Passes_Completed_5ma,Home_Through_Balls_ema5,Home_Through_Balls_ema38,Home_Through_Balls_5ma,Home_Through_Balls_Completed_ema5,Home_Through_Balls_Completed_ema38,Home_Through_Balls_Completed_5ma,Home_Tackles_Won_Completed_ema5,Home_Tackles_Won_Completed_ema38,Home_Tackles_Won_Completed_5ma,Away_Goals_ema5,Away_Goals_ema38,Away_Goals_5ma,Away_xG_ema5,Away_xG_ema38,Away_xG_5ma,Away_Deep_ema5,Away_Deep_ema38,Away_Deep_5ma,Away_ppda_ema5,Away_ppda_ema38,Away_ppda_5ma,Away_Possession_ema5,Away_Possession_ema38,Away_Possession_5ma,Away_Total_Shots_ema5,Away_Total_Shots_ema38,Away_Total_Shots_5ma,Away_Shots_On_Target_ema5,Away_Shots_On_Target_ema38,Away_Shots_On_Target_5ma,Away_Corners_ema5,Away_Corners_ema38,Away_Corners_5ma,Away_Saves_ema5,Away_Saves_ema38,Away_Saves_5ma,Away_Shots_Off_Target_ema5,Away_Shots_Off_Target_ema38,Away_Shots_Off_Target_5ma,Away_Shots_Inside_the_Box_ema5,Away_Shots_Inside_the_Box_ema38,Away_Shots_Inside_the_Box_5ma,Away_Shots_Outside_the_Box_ema5,Away_Shots_Outside_the_Box_ema38,Away_Shots_Outside_the_Box_5ma,Away_Touches_ema5,Away_Touches_ema38,Away_Touches_5ma,Away_Touches_in_the_opposition_box_ema5,Away_Touches_in_the_opposition_box_ema38,Away_Touches_in_the_opposition_box_5ma,Away_Blocks_ema5,Away_Blocks_ema38,Away_Blocks_5ma,Away_Interceptions_ema5,Away_Interceptions_ema38,Away_Interceptions_5ma,Away_Clearances_ema5,Away_Clearances_ema38,Away_Clearances_5ma,Away_Duels_Won_ema5,Away_Duels_Won_ema38,Away_Duels_Won_5ma,Away_Aerial_Duels_Won_ema5,Away_Aerial_Duels_Won_ema38,Away_Aerial_Duels_Won_5ma,Away_Passes_Completed_ema5,Away_Passes_Completed_ema38,Away_Passes_Completed_5ma,Away_Total_Crosses_ema5,Away_Total_Crosses_ema38,Away_Total_Crosses_5ma,Away_Total_Crosses_Completed_ema5,Away_Total_Crosses_Completed_ema38,Away_Total_Crosses_Completed_5ma,Away_Total_Passes_Completed_ema5,Away_Total_Passes_Completed_ema38,Away_Total_Passes_Completed_5ma,Away_Long

In [350]:
#Engar nan raðir. Þetta er þá tilbúið til að halda áfram í EDA og spá.
#vistum nú datasettið sem er tilbúið
pick_insert = open('../data/Epl_data_train_ready.pickle','wb')
pickle.dump(df_ready, pick_insert)
pick_insert.close()